### import

In [2]:
import pandas as pd 
import numpy as np
import math
import re

from icecream import ic
from tqdm import tqdm 

from utils.custom_utils import load_var, save_var
# from utils.search_address import RetrieveAddress, get_lat_long

In [3]:
df_hk_json = load_var("df_hk_json")
df_where = df_hk_json['where']
# df_addr = pd.read_csv('./address/address.csv')
df_addr = pd.read_parquet('./address/df_addr.parquet.gzip')

In [4]:
texts = df_hk_json.iloc[:]['text']

In [14]:
ra = RetrieveAddress(df_addr)

## Finding address in a target text

#### dataframe: df_filtered

In [5]:
mask = df_addr.map(
    lambda x: bool(
        re.search(
            r'\b{}'.format(str(x)),
            texts[2],
            re.IGNORECASE)))
df_filtered = df_addr[mask.any(axis=1)]
# df_level = df_addr[mask[f'lv3']]

1. 불필요한 열이 포함됨.
이렇게 모든 열에서 찾으면,  
'서울', '관악구' 처럼 1단계 2단계가 들어간 모든 열이 포함된다.  

2. 검색을 2번 함.
아래에서 추려진 이름으로 다시 한번 검색하는데,  
이것 역시 중복된 작업이다.  

.

당장은 해결하지 않더라도,  
퍼포먼스를 끌어 올리려면 수정해야 할 것이다.  

### TODO: making a set from the retrieved address

#### def: get_set_from_columns()

In [121]:
def get_set_from_columns(df:pd.DataFrame):
    result_set = set([])
    
    for index, row in df.iterrows():
        for value in row.values:
            if isinstance(value, str):
                if value not in result_set:
                    result_set.add(value)
    return result_set 

In [122]:
addr_name_set = get_set_from_columns(df_filtered)
addr_name_set

{'가락동',
 '가리봉동',
 '가산동',
 '가양동',
 '가회동',
 '갈월동',
 '갈현동',
 '강남구',
 '강동구',
 '강동동',
 '강북구',
 '강서구',
 '강일동',
 '개봉동',
 '개포동',
 '개화동',
 '거여동',
 '견지동',
 '경동',
 '경운동',
 '계동',
 '계산동1가',
 '계산동2가',
 '고덕동',
 '고척동',
 '공덕동',
 '공릉동',
 '공평동',
 '공항동',
 '과해동',
 '관동1가',
 '관동2가',
 '관동3가',
 '관수동',
 '관악구',
 '관철동',
 '관훈동',
 '광복동1가',
 '광복동2가',
 '광복동3가',
 '광장동',
 '광진구',
 '광희동1가',
 '광희동2가',
 '교남동',
 '교동',
 '교북동',
 '구기동',
 '구랑동',
 '구로구',
 '구로동',
 '구산동',
 '구수동',
 '구완동',
 '구의동',
 '군자동',
 '궁동',
 '궁정동',
 '권농동',
 '금동',
 '금천구',
 '금호동1가',
 '금호동2가',
 '금호동3가',
 '금호동4가',
 '길동',
 '길음동',
 '낙원동',
 '남가좌동',
 '남대문로1가',
 '남대문로2가',
 '남대문로3가',
 '남대문로4가',
 '남대문로5가',
 '남북동',
 '남산동',
 '남산동1가',
 '남산동2가',
 '남산동3가',
 '남성로',
 '남영동',
 '남외동',
 '남일동',
 '남창동',
 '남포동1가',
 '남포동2가',
 '남포동3가',
 '남포동4가',
 '남포동5가',
 '남포동6가',
 '남학동',
 '남현동',
 '내곡동',
 '내동',
 '내발산동',
 '내수동',
 '내자동',
 '냉천동',
 '노고산동',
 '노량진동',
 '노원구',
 '녹번동',
 '녹산동',
 '논현동',
 '누상동',
 '누하동',
 '눌차동',
 '능동',
 '다동',
 '다운동',
 '달성동',
 '답동',
 '답십리동',
 '당산동',
 '당산동1가',
 '당산동2가',
 '당산동3가',
 '당산

In [10]:
len(addr_name_set)

8

#### issue: '서울' is not found 

'서울' 은 왜 없지?

#### fix: lv0

In [12]:
df_addr.head(1)

,lv1,lv2,lv3,lv4,lv5
0,충청남도,천안시 동남구,대흥동,NaN,NaN


`df_addr = pd.read_parquet('./address/df_addr.parquet.gzip')`

이렇게 불러오지 않았다. 

In [17]:
len(addr_name_set)

689

그러니까 안 바꾸면 700여개씩이나 처리해야 됨.

### TODO: Counting the found address

In [18]:
keyword_counts = {keyword: 0 for keyword in addr_name_set}

for keyword in addr_name_set:
    pattern = r'\b{}'.format(re.escape(keyword))
    count = len(re.findall(pattern, texts[2], re.IGNORECASE))
    keyword_counts[keyword] += count

In [19]:
df_keyword_counts = pd.DataFrame.from_dict(keyword_counts, orient='index', columns=['count'])

In [20]:
df_keyword_counts[df_keyword_counts['count'] > 0]

,count
서울,4
관악구,1
행동,1


- 관악구는 서울에 포함된다.
- '행동'의 윗단계는 없다.
- '서울' 4번, '관악구'가 1번 나왔다는 뜻은,  
  '서울 관악구' 일 경우가 전남 '행동' 일 경우보다 크다는 것.

#### get_keyword_counts()

In [6]:
def get_keyword_counts(address_set):

    keyword_counts = {keyword: 0 for keyword in address_set}
    
    for keyword in address_set:
        pattern = r'\b{}'.format(re.escape(keyword))
        count = len(re.findall(pattern, texts[2], re.IGNORECASE))
        keyword_counts[keyword] += count
        
    return keyword_counts

## 윗단계 인식하기

### TODO: Retrieving non-zero values

In [29]:
keyword_counts.get(next(iter(keyword_counts)))

0

#### dict: filtered_dict

In [30]:
# [item if item.value > 0 for item in keyword_counts]

filtered_dict = {key: value for key, value in keyword_counts.items() if value > 0}

In [31]:
filtered_dict

{'서울': 4, '관악구': 1, '행동': 1}

### TODO: which level is it?

#### def: filter_address()

In [88]:
def filter_address(keyword:str, df_addr:pd.DataFrame):
    mask = df_addr.map(
        lambda x: bool(
            re.search(
                r'\b{}'.format(str(x)),
                keyword,
                re.IGNORECASE)))
    df_filtered = df_addr[mask.any(axis=1)]
    return df_filtered

In [36]:
df_fltr = filter_address('서울', df_addr)

In [37]:
df_fltr.info()

<class 'pandas.core.frame.DataFrame'>
Index: 683 entries, 27516 to 28198
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   lv0     683 non-null    object 
 1   lv1     683 non-null    object 
 2   lv2     683 non-null    object 
 3   lv3     683 non-null    object 
 4   lv4     0 non-null      object 
 5   lv5     0 non-null      float64
dtypes: float64(1), object(5)
memory usage: 37.4+ KB


In [38]:
df_fltr.head()

,lv0,lv1,lv2,lv3,lv4,lv5
27516,서울,서울특별시,종로구,청운동,None,NaN
27517,서울,서울특별시,종로구,신교동,None,NaN
27518,서울,서울특별시,종로구,궁정동,None,NaN
27519,서울,서울특별시,종로구,효자동,None,NaN
27520,서울,서울특별시,종로구,창성동,None,NaN


#### ask: how to know a column name of a keyword

In [42]:
food = {
    'item': ['apple', 'banana', 'cherry', 'onion', 'garlic', 'flour'],
    'stock': [10, 5, 7, 8, 12, 5]
}
df_food = pd.DataFrame(food)


In [43]:
# Define conditions and choices for the 'code' column
conditions = [
    df_food['item'].isin(['apple', 'banana', 'cherry']),
    df_food['item'].isin(['onion', 'garlic']),
    df_food['item'].isin(['flour'])
]
choices = ['fruit', 'vegetable', 'grain']

# Create the 'code' column based on conditions
df_food['code'] = np.select(conditions, choices, default='')

print(df_food)

     item  stock       code
0   apple     10      fruit
1  banana      5      fruit
2  cherry      7      fruit
3   onion      8  vegetable
4  garlic     12  vegetable
5   flour      5      grain


In [39]:
import pandas as pd

# Step 1: Create a DataFrame
data = {
    'item': ['apple', 'banana', 'cherry', 'onion', 'garlic', 'flour'],
    'stock': [10, 5, 7, 8, 12, 5],
    'code': ['fruit', 'fruit', 'fruit', 'vegetable', 'vegetable', 'grain']
}

df = pd.DataFrame(data)

# Step 2: Find the column where 'onion' is located
column_name = None
for col in df.columns:
    if 'onion' in df[col].values:
        column_name = col
        break

# Step 3: Print the column name
print(column_name)

item


#### apply.

In [41]:
found_set = set([])
for col in df_fltr.columns:
    if '서울' in df_fltr[col].values:
        if col not in found_set:
            found_set.add(col)

In [42]:
found_set

{'lv0'}

#### def: get_level()

In [87]:
def get_level(keyword:str, df_fltr:pd.DataFrame):
    found_set = set([])
    for col in df_fltr.columns:
        if keyword in df_fltr[col].values:
            if col not in found_set:
                found_set.add(col)
                
    if len(found_set) > 0:
        return found_set
    else:
        return None

In [59]:
for key, value in filtered_dict.items():
    res = get_level(key)
    if res:
        if len(res) == 1:
            print(f'{key}({value}): {next(iter(res))}')
        else:
            print(f'{key}({value}): {res}')
            

서울(4): lv0
관악구(1): lv2


In [60]:
res = get_level(next(iter(filtered_dict)))

In [65]:
res_iter = iter(res)
next(res_iter)

'lv0'

근데...  
왜 '행동'이 lv3 이라고 나오지 않았지?  

#### issue: not found '행동' in lv3

In [75]:
fltr_iter = iter(filtered_dict)

In [78]:
next_item = next(fltr_iter)
next_item

'행동'

In [79]:
res = get_level(next_item)
res

In [83]:
for col in df_fltr.columns:
    ic(col)
    if next_item in df_fltr[col].values:
        ic(col)

ic| col: 'lv0'
ic| col: 'lv1'
ic| col: 'lv2'
ic| col: 'lv3'
ic| col: 'lv4'
ic| col: 'lv5'


In [84]:
df_fltr.info()

<class 'pandas.core.frame.DataFrame'>
Index: 683 entries, 27516 to 28198
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   lv0     683 non-null    object 
 1   lv1     683 non-null    object 
 2   lv2     683 non-null    object 
 3   lv3     683 non-null    object 
 4   lv4     0 non-null      object 
 5   lv5     0 non-null      float64
dtypes: float64(1), object(5)
memory usage: 37.4+ KB


In [85]:
if next_item in df_fltr['lv3'].values:
    ic(next_item)

In [86]:
df_fltr['lv3'].values

array(['청운동', '신교동', '궁정동', '효자동', '창성동', '통의동', '적선동', '통인동', '누상동',
       '누하동', '옥인동', '체부동', '필운동', '내자동', '사직동', '도렴동', '당주동', '내수동',
       '세종로', '신문로1가', '신문로2가', '청진동', '서린동', '수송동', '중학동', '종로1가', '공평동',
       '관훈동', '견지동', '와룡동', '권농동', '운니동', '익선동', '경운동', '관철동', '인사동',
       '낙원동', '종로2가', '팔판동', '삼청동', '안국동', '소격동', '화동', '사간동', '송현동',
       '가회동', '재동', '계동', '원서동', '훈정동', '묘동', '봉익동', '돈의동', '장사동', '관수동',
       '종로3가', '인의동', '예지동', '원남동', '연지동', '종로4가', '효제동', '종로5가', '종로6가',
       '이화동', '연건동', '충신동', '동숭동', '혜화동', '명륜1가', '명륜2가', '명륜4가', '명륜3가',
       '창신동', '숭인동', '교남동', '평동', '송월동', '홍파동', '교북동', '행촌동', '구기동',
       '평창동', '부암동', '홍지동', '신영동', '무악동', '무교동', '다동', '태평로1가', '을지로1가',
       '을지로2가', '남대문로1가', '삼각동', '수하동', '장교동', '수표동', '소공동', '남창동', '북창동',
       '태평로2가', '남대문로2가', '남대문로3가', '남대문로4가', '남대문로5가', '봉래동1가', '봉래동2가',
       '회현동1가', '회현동2가', '회현동3가', '충무로1가', '충무로2가', '명동1가', '명동2가',
       '남산동1가', '남산동2가', '남산동3가', '저동1가', '충무로4가', '충무로5가', '인현동

In [89]:
get_level(next_item, df_filtered)

{'lv3'}

df_filtered 를 안쓰고,  
'서울' 필터링된 df_fltr 을 써서 그랬음.  

#### def: which_level()

In [238]:
def which_level(df_filtered:pd.DataFrame, filtered_dict:dict) -> dict:
    results = {}
    for key, value in filtered_dict.items():
        res = get_level(key, df_filtered)
        if res:
            results[key] = {
                'count': value,
                'level': list(res)
            }
    return results

In [247]:
which_level(df_filtered, filtered_dict)

{'서울': {'count': 4, 'level': ['lv0']},
 '관악구': {'count': 1, 'level': ['lv2']},
 '행동': {'count': 1, 'level': ['lv3']}}

### TODO: checking upper level

lv3, lv2 의 상위 레벨은 무엇이고,  
찾은 결과에 그 레벨이 있는지 확인하기.  

In [92]:
filtered_dict

{'서울': 4, '관악구': 1, '행동': 1}

In [94]:
df_filtered.head(2)

,lv0,lv1,lv2,lv3,lv4,lv5
11913,전남,전라남도,순천시,행동,None,NaN
27516,서울,서울특별시,종로구,청운동,None,NaN


In [ ]:
got_level = which_level()

In [109]:
for key, value in got_level.items():
    # level = got_level[item]['level']
    ic(key, value)
    for lv in value['level']:
        if lv != 'lv0' and lv != 'lv1':
            ic(lv)

ic| key: '서울', value: {'count': 4, 'level': ['lv0']}
ic| key: '관악구', value: {'count': 1, 'level': ['lv2']}
ic| lv: 'lv2'
ic| key: '행동', value: {'count': 1, 'level': ['lv3']}
ic| lv: 'lv3'


#### def: upper_address()

In [110]:
def upper_address(df_address:pd.DataFrame, keyword:str, level:str):
    mask = df_address.map(
        lambda x: bool(
            re.search(
                r'\b{}'.format(str(x)),
                keyword,
                re.IGNORECASE)))
    # df_filtered = df_address[mask.any(axis=1)]
    df_filtered = df_address[mask[level]]
    return df_filtered

#### dict: level_int

In [113]:
level_int = {
    'lv0': 0,
    'lv1': 1, 
    'lv2': 2, 
    'lv3': 3
}

#### def: get_set_from_columns()

In [158]:
def get_set_from_columns(df:pd.DataFrame):
    result_set = set([])
    
    for index, row in df.iterrows():
        for col, value in row.items():
            # ic(col, value)
            if isinstance(value, str):
                if value not in result_set:
                    result_set.add(value)
                    
    return result_set 

In [157]:
for key, value in got_level.items():
    # level = got_level[item]['level']
    ic(key, value)
    for lv in value['level']:
        if lv != 'lv0' and lv != 'lv1':
            df_upper = upper_address(df_addr, key, lv)
            result_set = get_set_from_columns(df_upper)
            ic(result_set)

ic| key: '서울', value: {'count': 4, 'level': ['lv0']}
ic| key: '관악구', value: {'count': 1, 'level': ['lv2']}
ic| result_set: {'관악구', '신림동', '서울', '봉천동', '서울특별시', '남현동'}
ic| key: '행동', value: {'count': 1, 'level': ['lv3']}
ic| result_set: {'순천시', '행동', '전남', '전라남도'}


상위 레벨 주소가 무엇인지 확인하기.  
- 결과의 모든 레벨 컬럼의 내용을 리스트로 만든다.  
- 현재 주소의 한단계 위의 주소가 무엇인지 확인한다.  

In [146]:
result_set = set([])
result_dict = {}

Series — pandas 2.2.2 documentation (pydata.org) | [Link](https://pandas.pydata.org/pandas-docs/stable/reference/series.html#series) | `w21.2024-05-17.Friday, 12:25:09`

In [147]:
for index, row in df_upper.iterrows():
    for col, value in row.items():
        # ic(col, value)
        if isinstance(value, str):
            if value not in result_set:
                result_set.add(value)
                result_dict[value] = col

In [149]:
ic(result_set)
ic(result_dict)

ic| result_set: {'순천시', '행동', '전남', '전라남도'}
ic| result_dict: {'순천시': 'lv2', '전남': 'lv0', '전라남도': 'lv1', '행동': 'lv3'}


{'전남': 'lv0', '전라남도': 'lv1', '순천시': 'lv2', '행동': 'lv3'}

#### def: get_dict_from_columns

In [182]:
def get_dict_from_columns(df:pd.DataFrame):
    result_set = set([])
    result_dict = {}
    
    for index, row in df.iterrows():
        for col, value in row.items():
            # ic(col, value)
            if isinstance(value, str):
                if value not in result_set:
                    result_set.add(value)
                    result_dict[value] = col
                    
    return result_dict 

In [155]:
get_set_from_columns(df_upper)

{'전남': 'lv0', '전라남도': 'lv1', '순천시': 'lv2', '행동': 'lv3'}

이렇게 하면 안되네?  

- '행동' 을 함수에 넣는다.
- '행동' 의 한단계 위의 주소를 건네준다.

#### solution 1

In [159]:
df_filtered.head(2)

,lv0,lv1,lv2,lv3,lv4,lv5
11913,전남,전라남도,순천시,행동,None,NaN
27516,서울,서울특별시,종로구,청운동,None,NaN


In [162]:
next_item = iter(filtered_dict)

In [163]:
item = next(next_item)

In [168]:
addrs = [key for key, value in filtered_dict.items()]

In [169]:
addrs[0]

'서울'

In [208]:
item_curr = addrs[1]
item_curr

'관악구'

In [212]:
df_item = filter_address(item_curr, df_addr)

In [213]:
df_item.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3 entries, 28150 to 28152
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   lv0     3 non-null      object 
 1   lv1     3 non-null      object 
 2   lv2     3 non-null      object 
 3   lv3     3 non-null      object 
 4   lv4     0 non-null      object 
 5   lv5     0 non-null      float64
dtypes: float64(1), object(5)
memory usage: 168.0+ bytes


In [214]:
df_item.head()

,lv0,lv1,lv2,lv3,lv4,lv5
28150,서울,서울특별시,관악구,봉천동,None,NaN
28151,서울,서울특별시,관악구,신림동,None,NaN
28152,서울,서울특별시,관악구,남현동,None,NaN


In [215]:
item_levels = which_level()
item_levels

{'서울': {'count': 4, 'level': ['lv0']},
 '관악구': {'count': 1, 'level': ['lv2']},
 '행동': {'count': 1, 'level': ['lv3']}}

In [216]:
item_lv = item_levels[item_curr]['level'][0]
item_lv

'lv2'

In [217]:
item_col = get_dict_from_columns(df_item)
item_col

{'서울': 'lv0',
 '서울특별시': 'lv1',
 '관악구': 'lv2',
 '봉천동': 'lv3',
 '신림동': 'lv3',
 '남현동': 'lv3'}

In [218]:
item_reversed = {value: key for key, value in item_col.items()}
item_reversed

{'lv0': '서울', 'lv1': '서울특별시', 'lv2': '관악구', 'lv3': '남현동'}

In [220]:
for key in filtered_dict.keys():
    for k, v in item_reversed.items():
        if k != item_lv:
            if key == v:
                ic(v)

ic| v: '서울'


됐다!!!  
이제 함수로 만들면 된다.  

#### def: isexistance_upper()

In [241]:
def isexistance_upper(
    keyword:str,
    df_address:pd.DataFrame,
    filtered_dict:dict,
    verbose=False
):
    if verbose: ic(keyword)

    existance = False

    df_item = filter_address(keyword, df_addr)
    item_col = get_dict_from_columns(df_item)
    item_reversed = {value: key for key, value in item_col.items()}
    if verbose: ic(item_reversed)
    
    item_levels = which_level(df_filtered=df_filtered, filtered_dict=filtered_dict)
    item_lv = item_levels[keyword]['level'][0]
    if verbose: ic(item_lv)

    for key in filtered_dict.keys():
        for k, v in item_reversed.items():
            if k != item_lv or k == 'lv0':
                if key == v:
                    existance = True
                    if verbose: ic(v)
                
    return existance

In [242]:
isexistance_upper(addrs[2], df_addr, filtered_dict, verbose=True)

ic| keyword: '행동'
ic| item_reversed: {'lv0': '전남', 'lv1': '전라남도', 'lv2': '순천시', 'lv3': '행동'}
ic| item_lv: 'lv3'


False

### TODO: verifying an existance of each address

이제 비교.  
filtered_dict 아이템 하나씩 비교.  
isexistance_upper() 로 비교.  

#### try: filtered_dict with isexistance_upper()

In [243]:
results = []
for key, value in filtered_dict.items():
    if isexistance_upper(key, df_addr, filtered_dict, verbose=True):
        results.append(key)

ic| keyword: '서울'
ic| item_reversed: {'lv0': '서울', 'lv1': '서울특별시', 'lv2': '강동구', 'lv3': '강일동'}
ic| item_lv: 'lv0'
ic| v: '서울'
ic| keyword: '관악구'
ic| item_reversed: {'lv0': '서울', 'lv1': '서울특별시', 'lv2': '관악구', 'lv3': '남현동'}
ic| item_lv: 'lv2'
ic| v: '서울'
ic| keyword: '행동'
ic| item_reversed: {'lv0': '전남', 'lv1': '전라남도', 'lv2': '순천시', 'lv3': '행동'}
ic| item_lv: 'lv3'


In [244]:
results

['서울', '관악구']

In [246]:
levels = which_level(df_filtered=df_filtered, filtered_dict=filtered_dict)
levels

{'서울': {'count': 4, 'level': ['lv0']},
 '관악구': {'count': 1, 'level': ['lv2']},
 '행동': {'count': 1, 'level': ['lv3']}}

#### def: get_existance()

In [280]:
def get_existance(
    df_address:pd.DataFrame,
    filtered_dict:dict,
    verbose=False
):
    results = []
    for key, value in filtered_dict.items():
        if isexistance_upper(key, df_address, filtered_dict, verbose=verbose):
            results.append(key)
            
    return results

In [281]:
existances = get_existance(df_addr, filtered_dict, verbose=True)

ic| keyword: '서울'
ic| item_reversed: {'lv0': '서울', 'lv1': '서울특별시', 'lv2': '강동구', 'lv3': '강일동'}
ic| item_lv: 'lv0'
ic| v: '서울'
ic| keyword: '관악구'
ic| item_reversed: {'lv0': '서울', 'lv1': '서울특별시', 'lv2': '관악구', 'lv3': '남현동'}
ic| item_lv: 'lv2'
ic| v: '서울'
ic| keyword: '행동'
ic| item_reversed: {'lv0': '전남', 'lv1': '전라남도', 'lv2': '순천시', 'lv3': '행동'}
ic| item_lv: 'lv3'


In [282]:
existances

['서울', '관악구']

#### dict: existed_levels

In [250]:
existed_levels = {}
for item in results:
    lv = levels[item]['level']
    if len(lv) == 1:
        lv_int = level_int[lv[0]]
        ic(lv_int)
        existed_levels[item] = lv_int
existed_levels

ic| lv_int: 0
ic| lv_int: 2


{'서울': 0, '관악구': 2}

In [257]:
max_level = -1
sorted_level = []
for key, value in existed_levels.items():
    item = {key: value}
    if value > max_level:
        max_level = value
        sorted_level.insert(0, item)
        ic(sorted_level)
    else:
        sorted_level.append(item)

ic| sorted_level: [{'서울': 0}]
ic| sorted_level: [{'관악구': 2}, {'서울': 0}]


In [258]:
sorted_level

[{'관악구': 2}, {'서울': 0}]

In [268]:
list(sorted_level[0].keys())[0]

'관악구'

#### def: get_existed_levels()

In [289]:
def get_existed_levels(existances:dict, level_int:dict):
    existed_levels = {}
    for item in existances:
        lv = levels[item]['level']
        if len(lv) == 1:
            lv_int = level_int[lv[0]]
            # ic(lv_int)
            existed_levels[item] = lv_int
    return existed_levels

In [290]:
existed_levels = get_existed_levels(existances, level_int)

In [291]:
existed_levels

{'서울': 0, '관악구': 2}

#### def: max_address()

In [296]:
def top_address(existed_levels:dict):

    max_level = -1
    sorted_level = []
    for key, value in existed_levels.items():
        item = {key: value}
        if value > max_level:
            max_level = value
            sorted_level.insert(0, item)
            # ic(sorted_level)
        else:
            sorted_level.append(item)
            
    return list(sorted_level[0].keys())[0]

In [297]:
top_address(existed_levels)

'관악구'

## Process

- df_addr
- df_filtered <-- df_addr
- get_keyword_counts() <-- get_set_from_columns()
- filtered_dict <-- keyword_counts
    - get_keyword_counts()
- `addrs = [key for key, value in filtered_dict.items()]`
- get_set_from_columns() <-- df_filtered
- keyword_counts <-- get_set_from_columns()
- filter_address(keyword:str, df_addr:pd.DataFrame)
- df_fltr
    - filter_address(keyword:str, df_addr:pd.DataFrame)
- get_level(keyword:str, df_fltr:pd.DataFrame)
- which_level()
    - get_level(key, df_filtered)
- get_dict_from_columns()
- filter_address()
- is_existance_upper() <-- addrs[], df_addr, filtered_dict
    - filter_address()
    - get_dict_from_columns()
    - which_level(df_filtered, filtered_dict)
- get_existance() <-- df_addr, filtered_dict
    - is_existance_upper()
- existance <-- get_existance()
- level_int <-- dict
- get_existed_levels() <-- existance, level_int
- existed_levels <-- get_existed_levels()
- top_address() <-- existed_levels

In [1]:
import pandas as pd 
import numpy as np
import math
import re

from icecream import ic
from tqdm import tqdm 

from utils.custom_utils import load_var, save_var
# from utils.search_address import RetrieveAddress, get_lat_long

df_hk_json = load_var("df_hk_json")
df_where = df_hk_json['where']
# df_addr = pd.read_csv('./address/address.csv')
df_addr = pd.read_parquet('./address/df_addr.parquet.gzip')

texts = df_hk_json.iloc[:]['text']

In [2]:
text = texts[2]

In [3]:
def get_filtered(df_address:pd.DataFrame, text):
    mask = df_address.map(
        lambda x: bool(
            re.search(
                r'\b{}'.format(str(x)),
                text,
                re.IGNORECASE)))
    df_filtered = df_address[mask.any(axis=1)]
    return df_filtered

In [4]:
df_filtered = get_filtered(df_addr, text)

In [5]:
def get_set_from_columns(df:pd.DataFrame):
    result_set = set([])
    
    for index, row in df.iterrows():
        for value in row.values:
            if isinstance(value, str):
                if value not in result_set:
                    result_set.add(value)
    return result_set 

In [6]:
address_set = get_set_from_columns(df_filtered)

In [7]:
def get_keyword_counts(address_set: set, text: str):
    # Initialize the dictionary with keywords and their counts set to 0
    keyword_counts = {keyword: 0 for keyword in address_set}

    # Iterate through each keyword in the set
    for keyword in address_set:
        # Create a regex pattern with word boundaries
        pattern = r'\b{}'.format(re.escape(keyword))
        # Find all matches of the pattern in the text (case-insensitive)
        regex_result = re.findall(pattern, text, flags=re.IGNORECASE)
        # Count the number of matches
        count = len(regex_result)
        # Update the count for the keyword in the dictionary
        keyword_counts[keyword] += count

    return keyword_counts

In [ ]:
keyword_counts = get_keyword_counts(address_set=address_set, text=text)
filtered_dict = {key: value for key, value in keyword_counts.items() if value > 0}

In [10]:
def get_level(keyword:str, df_fltr:pd.DataFrame):
    found_set = set([])
    for col in df_fltr.columns:
        if keyword in df_fltr[col].values:
            if col not in found_set:
                found_set.add(col)
                
    if len(found_set) > 0:
        return found_set
    else:
        return None

In [11]:
def which_level(df_filtered:pd.DataFrame, filtered_dict:dict) -> dict:
    results = {}
    for key, value in filtered_dict.items():
        res = get_level(key, df_filtered)
        if res:
            results[key] = {
                'count': value,
                'level': list(res)
            }
    return results

In [12]:
levels = which_level(df_filtered=df_filtered, filtered_dict=filtered_dict)

In [13]:
def filter_address(keyword:str, df_addr:pd.DataFrame):
    mask = df_addr.map(
        lambda x: bool(
            re.search(
                r'\b{}'.format(str(x)),
                keyword,
                re.IGNORECASE)))
    df_filtered = df_addr[mask.any(axis=1)]
    return df_filtered

In [14]:
def get_dict_from_columns(df:pd.DataFrame):
    result_set = set([])
    result_dict = {}
    
    for index, row in df.iterrows():
        for col, value in row.items():
            # ic(col, value)
            if isinstance(value, str):
                if value not in result_set:
                    result_set.add(value)
                    result_dict[value] = col
                    
    return result_dict 

In [15]:
def isexistance_upper(
    keyword:str,
    df_address:pd.DataFrame,
    filtered_dict:dict,
    levels:dict,
    verbose=False
):
    if verbose: ic(keyword)

    existance = False

    df_item = filter_address(keyword, df_addr)
    item_col = get_dict_from_columns(df_item)
    item_reversed = {value: key for key, value in item_col.items()}
    if verbose: ic(item_reversed)
    
    item_lv = levels[keyword]['level'][0]
    if verbose: ic(item_lv)

    for key in filtered_dict.keys():
        for k, v in item_reversed.items():
            if k != item_lv or k == 'lv0':
                if key == v:
                    existance = True
                    if verbose: ic(v)
                
    return existance

In [16]:
def get_existance(
    df_address:pd.DataFrame,
    filtered_dict:dict,
    levels:dict,
    verbose=False
):
    results = []
    for key, value in filtered_dict.items():
        if isexistance_upper(key, df_address, filtered_dict, levels=levels, verbose=verbose):
            results.append(key)
            
    return results

In [17]:
existances = get_existance(df_addr, filtered_dict, levels)

In [18]:
level_int = {
    'lv0': 0,
    'lv1': 1, 
    'lv2': 2, 
    'lv3': 3
}

In [19]:
def get_existed_levels(existances:dict, level_int:dict, levels:dict):
    existed_levels = {}
    for item in existances:
        lv = levels[item]['level']
        if len(lv) == 1:
            lv_int = level_int[lv[0]]
            # ic(lv_int)
            existed_levels[item] = lv_int
    return existed_levels

In [20]:
existed_levels = get_existed_levels(existances, level_int, levels)

In [21]:
def top_address(existed_levels:dict):

    max_level = -1
    sorted_level = []
    for key, value in existed_levels.items():
        item = {key: value}
        if value > max_level:
            max_level = value
            sorted_level.insert(0, item)
            # ic(sorted_level)
        else:
            sorted_level.append(item)
            
    return list(sorted_level[0].keys())[0]

In [22]:
top_address(existed_levels)

'관악구'

## Test - CheckAddressLevels

In [1]:
import pandas as pd 
import numpy as np
import math
import re

from icecream import ic
from tqdm import tqdm 

from utils.custom_utils import load_var, save_var
from utils.search_address import RetrieveAddress, get_lat_long, CheckAddressLevels

df_hk_json = load_var("df_hk_json")
df_where = df_hk_json['where']
# df_addr = pd.read_csv('./address/address.csv')
df_addr = pd.read_parquet('./address/df_addr.parquet.gzip')

texts = df_hk_json.iloc[:]['text']

In [16]:
text = texts[5]
text

'강남역의 \'헌팅 포차\' 거리 한 술집에 야외 테이블까지 사람들이 가득 차 있는 모습. /사진=김세린 기자\n23일 저녁, \'헌팅포차\'가 밀집한 서울 강남역 인근의 거리는 젊은 남녀들로 북적였다. 지난 11일 정부가 3년 4개월 만에 사실상 \'코로나19 엔데믹(풍토병으로 굳어진 감염병)\'을 선언한 가운데, 서울 강남 인근에서 이른바 \'헌팅 메카\'로 불리는 곳들은 술을 마시고 즐기려는 인파가 가득했다.\n\n"헌팅 술집은 원래 자연스럽게 다른 분들이랑 합석해서 즐기는 분위기잖아요. 모르는 사람들이랑 술 마시고 얘기하는 것 자체가 재미있고, 마음에 드는 이성을 찾을 수도 있어서 좋죠"\n\n코로나19 팬데믹 이후 오랜만에 \'헌팅포차\'를 찾게 됐다는 대학생 김모 씨(22)는 "작년까지만 해도 실내 마스크도 착용해야 하고 해서 굳이 이 거리를 오지 않았다"면서도 "평일 밤인데도 사람이 많아서 북적이는 분위기가 좋다"며 이같이 말했다.\n\n\n화요일에도 뜨거운 밤의 열기…헌팅 술집 \'만석\'\n강남역의 한 클럽 앞에 사람들이 입장을 위해 대기줄을 서 있는 모습. /사진=김세린 기자\n몇몇 헌팅포차 앞에는 손님 수십명이 줄지어 입장을 대기하고 있었다. 야외 테라스석을 포함한 모든 자리가 만석인 탓이다. 오랜 시간 이곳을 대기하던 사람들은 지나가는 다른 무리를 붙잡고 "함께 술집에 들어가자"고 즉석 헌팅을 하기도 했다. 이 헌팅포차 직원은 "30~40분은 대기해야 입장할 수 있다"며 "다른 테이블과 합석하면 더 빠른 입장이 가능하다"고 안내했다.\n\n인근의 다른 술집도 마찬가지였다. 코로나 기간 한창 공사 중이었던 한 유명 클럽도 젊은 층이 모여 다시 활기를 띠고 있었다. 해당 클럽 앞 관계자는 "오픈한지 50분도 안 지났는데 벌써 50명이 넘게 안에 모여있다"며 "시간이 지나면 더 들어올 것 같은데, 클럽 안에 관계자라든지 아는 분 계시면 더 빠른 입장이 가능하다"고 전했다.\n\n자정이 가까워오자 헌팅 술집과 클럽 인근 거리에서 본격적인 \'헌팅

In [17]:
cal = CheckAddressLevels(df_address=df_addr, text=text)

In [20]:
addr = cal.process()
addr

'서울'

In [11]:
get_lat_long(addr)

(37.5300432, 127.0313346)

> 강남역, 압구정, 압구정로데오 ...

주소는 아니지만 자주쓰이는 위치정보인데, 미처 생각하지 못했다.  
정확한 주소명으로 찾아내는 방식은 이런 한계점이 있다.  
그렇다고 전철역, 기차역을 다 넣어야하나...  

> '거리'

이건 또 무슨 ... ㅋㅋㅋ  

## Blogging

In [23]:
import pandas as pd 
import numpy as np
import math
import re

from icecream import ic
from tqdm import tqdm 

from utils.custom_utils import load_var, save_var
# from utils.search_address import RetrieveAddress, get_lat_long

df_hk_json = load_var("df_hk_json")
df_where = df_hk_json['where']
# df_addr = pd.read_csv('./address/address.csv')
df_addr = pd.read_parquet('./address/df_addr.parquet.gzip')

texts = df_hk_json.iloc[:]['text']

In [24]:
text = texts[2]
text

'/사진=유튜브 채널 \'서울경찰\' 영상 캡처\n출근길 \'묻지마 폭행\'을 저지른 남성이 현행범으로 체포되는 모습이 공개됐다.\n\n18일 서울경찰청 공식 유튜브 채널에 \'출근길 묻지마 폭행범 검거 현장\'이라는 제목으로 지난 3월 서울 관악구 골목길에서 난동을 부렸던 남성의 모습이 담긴 영상이 게재됐다.\n\n이 남성은 상의를 벗어 문신이 그려진 몸으로 길을 걷던 한 여성을 향해 달려갔다. 갑자기 돌진하는 남성을 보며 놀란 여성이 횡단보도로 뛰어가며 도망쳤지만, 남성은 그 뒤를 쫓아갔다.\n\n다른 행인들이 남성을 말렸음에도 난동은 이어졌다. 신고받고 현장에 출동한 경찰은 폭행 및 공무집행방해 등 혐의로 현장에서 체포했다.\n/사진=유튜브 채널 \'서울경찰\' 영상 캡처\n경찰의 등장에도 남성은 침을 뱉고, 욕을 하는 등 폭력적인 행동을 이어갔다. 결국 경찰은 남성을 힘으로 제압해 압송했다.\n\n경찰에 따르면 이 남성과 여성은 일면식도 없는 사이였다. 경찰은 "마약사범으로 의심했으나 마약 성분은 검출되지 않았다"며 "관제센터와의 공조, 시민들의 도움으로 빠르게 찾아낸 덕에 2차 피해를 막을 수 있었다"고 전했다.\n\n김소연 한경닷컴 기자 sue123@hankyung.com'

In [25]:
cal = CheckAddressLevels(df_address=df_addr, text=text)

In [28]:
cal.df_filtered.head()

,lv0,lv1,lv2,lv3,lv4,lv5
11913,전남,전라남도,순천시,행동,None,NaN
27516,서울,서울특별시,종로구,청운동,None,NaN
27517,서울,서울특별시,종로구,신교동,None,NaN
27518,서울,서울특별시,종로구,궁정동,None,NaN
27519,서울,서울특별시,종로구,효자동,None,NaN


In [29]:
cal.address_set

{'가락동',
 '가리봉동',
 '가산동',
 '가양동',
 '가회동',
 '갈월동',
 '갈현동',
 '강남구',
 '강동구',
 '강동동',
 '강북구',
 '강서구',
 '강일동',
 '개봉동',
 '개포동',
 '개화동',
 '거여동',
 '견지동',
 '경동',
 '경운동',
 '계동',
 '계산동1가',
 '계산동2가',
 '고덕동',
 '고척동',
 '공덕동',
 '공릉동',
 '공평동',
 '공항동',
 '과해동',
 '관동1가',
 '관동2가',
 '관동3가',
 '관수동',
 '관악구',
 '관철동',
 '관훈동',
 '광복동1가',
 '광복동2가',
 '광복동3가',
 '광장동',
 '광진구',
 '광희동1가',
 '광희동2가',
 '교남동',
 '교동',
 '교북동',
 '구기동',
 '구랑동',
 '구로구',
 '구로동',
 '구산동',
 '구수동',
 '구완동',
 '구의동',
 '군자동',
 '궁동',
 '궁정동',
 '권농동',
 '금동',
 '금천구',
 '금호동1가',
 '금호동2가',
 '금호동3가',
 '금호동4가',
 '길동',
 '길음동',
 '낙원동',
 '남가좌동',
 '남대문로1가',
 '남대문로2가',
 '남대문로3가',
 '남대문로4가',
 '남대문로5가',
 '남북동',
 '남산동',
 '남산동1가',
 '남산동2가',
 '남산동3가',
 '남성로',
 '남영동',
 '남외동',
 '남일동',
 '남창동',
 '남포동1가',
 '남포동2가',
 '남포동3가',
 '남포동4가',
 '남포동5가',
 '남포동6가',
 '남학동',
 '남현동',
 '내곡동',
 '내동',
 '내발산동',
 '내수동',
 '내자동',
 '냉천동',
 '노고산동',
 '노량진동',
 '노원구',
 '녹번동',
 '녹산동',
 '논현동',
 '누상동',
 '누하동',
 '눌차동',
 '능동',
 '다동',
 '다운동',
 '달성동',
 '답동',
 '답십리동',
 '당산동',
 '당산동1가',
 '당산동2가',
 '당산동3가',
 '당산

In [31]:
cal.filtered_dict

{'서울': 4, '행동': 1, '관악구': 1}

In [32]:
cal.levels

{'서울': {'count': 4, 'level': ['lv0']},
 '행동': {'count': 1, 'level': ['lv3']},
 '관악구': {'count': 1, 'level': ['lv2']}}

In [33]:
cal.existances

['서울', '관악구']

In [34]:
cal.existed_levels

{'서울': 0, '관악구': 2}

In [26]:
addr = cal.process()
addr

'관악구'

In [27]:
get_lat_long(addr)

(37.4782, 126.9518)

## Test - Results Inspection

### import

In [29]:
import pandas as pd 
import numpy as np
import math
import re

from icecream import ic
from tqdm.notebook import tqdm 

from utils.custom_utils import load_var, save_var
from utils.search_address import RetrieveAddress, get_lat_long, CheckAddressLevels
from utils.webscraping import ArticleScraper

In [30]:
from importlib import reload

In [2]:
df_addr = pd.read_parquet('./address/df_addr.parquet.gzip')
df_text = pd.read_parquet('./dataframe/flasher_hk_20130101_20220307.gzip')

### todo: extracting address with the class

#### filtering empty contents

In [13]:
df_filtered = df_text[df_text['content'].notna()]

In [14]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 148 entries, 0 to 271
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   범죄 유형          0 non-null      float64
 1   지역             0 non-null      float64
 2   위도             0 non-null      float64
 3   경도             0 non-null      float64
 4   중복 여부          0 non-null      float64
 5   퀄리티            0 non-null      float64
 6   제외 여부          0 non-null      float64
 7   기사제목           148 non-null    object 
 8   사건 장소          125 non-null    object 
 9   수사 기관          148 non-null    object 
 10  본문             148 non-null    object 
 11  URL            148 non-null    object 
 12  일자             148 non-null    int64  
 13  언론사            148 non-null    object 
 14  기고자            144 non-null    object 
 15  비고             0 non-null      float64
 16  selenium_html  148 non-null    object 
 17  content        148 non-null    object 
dtypes: float64(8), 

In [ ]:
columns = list(df_text.columns)

In [20]:
columns[1]

'지역'

In [15]:
index_filtered = df_filtered.index

In [23]:
df_text.loc[:, columns[1]] = ''

#### reload for debugging

In [31]:
reload(CheckAddressLevels)

ImportError: module CheckAddressLevels not in sys.modules

#### autoreload

In [35]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
%aimport utils.search_address.CheckAddressLevels

ModuleNotFoundError: No module named 'utils.search_address.CheckAddressLevels'; 'utils.search_address' is not a package

In [36]:
from utils.search_address import CheckAddressLevels

In [38]:
%autoreload 2

In [47]:
for index in tqdm(index_filtered[7:]):
    text = df_filtered.loc[index, 'content']
    cal = CheckAddressLevels(df_address=df_addr, text=text)
    addr = cal.process()
    df_text.loc[index, columns[1]] = str(addr)

  0%|          | 0/141 [00:00<?, ?it/s]

150개에 8분.  
100개에 2분40초. --> 대략 3분.  

개당 3~5초.  
10개 40초.  
100개 400초. -->   6분40초. 7분.  

#### issue: empty data

In [44]:
text

'\n\n\n\n\n/사진=유튜브 승무원 룩북 영상 캡처\n\'승무원 룩북\' 영상으로 논란이 불거진 유튜버가 자신이 입던 속옷을 나눠준다는 이벤트를 벌였다는 주장이 나왔다. 최근 온라인 커뮤니티에 \'승무원 룩북\' 동영상을 자신의 유튜브 채널에 게재했던 유튜버 A 씨가 미국의 아프리카 TV로 불리는 페트리온 유료 회원들을 대상으로 \'속옷 나눔 이벤트\'를 진행했다고 주장하는 글이 게재됐다. 작성자는 A 씨의 속옷 사진을 올리면서 "이벤트 형식으로 구독자에게 입던 속옷과 스타킹을 나눔하려 했고, 이후에는 판매하려고 했다"고 전했다. A 씨는 유튜브 채널에서 속옷 차림으로 등장해 옷을 갈아 입는 동영상을 게재해 왔다. 하지만 해당 콘텐츠 상세보기 페이지를 통해 페트리온 접속 주소를 홍보했고, 해당 페이지를 통해 10달러(약 1만2000원)에서 600달러(약 72만 원)까지 후원 금액에 따라 노출 영상을 차등 공개해 왔다. 특히 가장 많은 후원금을 내는 VVIP 회원에게는 수위가 적나라한 노출 동영상을 공개한 것으로 알려졌다.        승무원 룩북 영상을 게재한 후 특정 직업군 성상품화 논란에 휩싸였던 A 씨는 "채널에 게시한 영상 중 일부가 무단으로 캡처되어 특정 커뮤니티의 게시판에 악의적인 제목 및 내용으로 게시된 사실을 인지하였고, 해당 게시글에 달린 수천여 개의 댓글 중 욕설을 포함한 모욕적인 표현 등이 담긴 악성 댓글을 작성한 100여 명을 명예훼손죄, 모욕죄 등의 혐의로 고소하는 절차를 진행하고 있다"고 밝혔다. 하지만 이후 유튜버 구제역은 21일 자신의 유튜브 채널을 통해 A 씨가 패트리온 영상을 홍보하고 있다고 주장하면서 "A 씨가 현재 하는 행동은 성 상품화가 맞다. 남녀노소가 보는 유튜브에서까지 자신의 성적인 영상을 홍보하는 건 잘못됐다"면서 성매매특별법과 정보통신망법 위반 혐의로 고발했다고 밝혔다. 정보통신망법 44조7항에 따르면 정보통신망을 통해 음란한 부호, 문언, 음향, 화상 또는 영상을 배포, 판매, 임대하거나 전시하면 1년 이하

In [43]:
cal.df_filtered

,lv0,lv1,lv2,lv3,lv4,lv5
11913,전남,전라남도,순천시,행동,None,NaN


주소명이 없을 때는 프로세스를 멈추고, None 리턴

#### inspection: result

In [61]:
df_res = df_text[df_text[columns[1]].notna() & (df_text[columns[1]] != None) & (df_text[columns[1]] != "") & (df_res[columns[1]] != 'None')]

In [62]:
df_res.info()

<class 'pandas.core.frame.DataFrame'>
Index: 92 entries, 0 to 271
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   범죄 유형          0 non-null      float64
 1   지역             92 non-null     object 
 2   위도             0 non-null      float64
 3   경도             0 non-null      float64
 4   중복 여부          0 non-null      float64
 5   퀄리티            0 non-null      float64
 6   제외 여부          0 non-null      float64
 7   기사제목           92 non-null     object 
 8   사건 장소          82 non-null     object 
 9   수사 기관          92 non-null     object 
 10  본문             92 non-null     object 
 11  URL            92 non-null     object 
 12  일자             92 non-null     int64  
 13  언론사            92 non-null     object 
 14  기고자            88 non-null     object 
 15  비고             0 non-null      float64
 16  selenium_html  92 non-null     object 
 17  content        92 non-null     object 
dtypes: float64(7), i

In [67]:
for index in df_res.index:
    print(index, df_res.loc[index, columns[1]])

0 서울
1 서울특별시
2 대전
4 관리
5 경기
7 대구
8 서울
9 제주
10 강원
11 부평구
12 서울
13 서울
14 제주
16 부평구
17 경북
18 서울
22 부산
23 경기
24 대덕구
25 부평구
26 충남
27 서울
28 부평구
29 부평구
30 서울
31 옥련동
32 부평구
33 관악구
34 제주시
35 광산구
36 광주
37 서울
40 서울
44 경북
45 중계동
46 노원구
52 하남시
53 경기
54 서울
56 김제시
57 용산구
58 서울
60 경기
61 서울
64 인천
68 부산
69 서울
70 인천
71 서울
74 대전
75 인천
77 서울
78 서울
80 인천
81 인천
84 서울
86 화성시
87 화성시
88 수원시 장안구
89 경북
90 안성시
91 안성시
93 부산
94 서울
96 서울
97 서울
99 대전
100 삼산동
102 서울
103 남구
106 화성시
109 처리
110 경기
111 경기
112 복대동
113 경기
114 경기
119 연남동
133 서산시
134 전남
135 광진구
261 부평구
262 부평구
263 부평구
264 부평구
265 인천
266 부평동
267 부평구
268 부평구
269 부평구
270 부평구
271 부평구


#### issue: not full content in Series 

In [109]:
df_res.iloc[3]['content']

'\n\n\n\n\n가출 여고생에게 숙식을 제공하며 성매매를 강요한 일당이 실형을 선고받았다. 이들 중 한 명은 공범 몰래 여고생을 성폭행하기도 했다. 사진은 기사와 무관함. /사진=게티이미지뱅크 \n가출 여고생에게 숙식을 제공하며 성매매를 강요한 일당이 실형을 선고받았다. 이들 중 한 명은 공범 몰래 여고생을 성폭행하기도 했다. 대구지방법원 포항지원 제1형사부(권순향 재판장)는 아동청소년의성보호에관한법률위반(강요행위) 등의 혐의로 기소된 A씨(22), B씨(25), C씨(22·여)에게 각각 징역 6년, 징역 5년, 징역 4년을 선고했다고 19일 밝혔다.         재판부는 이들에게 80시간의 성폭력 치료프로그램 이수와 7년간 아동·청소년·장애인 취업 제한도 함께 명령했다. A씨 등은 2019년 11월부터 2020년 9월까지 고향 후배인 D양(17)에게 숙식을 제공하면서 성매매를 강요한 혐의를 받고 있다. 이들은 월 평균 25회, 하루 3~4회에 걸쳐 휴대폰 애플리케이션 등으로 성매수남을 모집해 D양에게 성매매를 강제했다. 또 D양이 성매매 대가로 받아돈 금액 중 70%를 갈취했다.        연인 관계인 A씨와 C씨는 2020년 8월 D양이 경기 인천으로 도망치자 "전화를 받지 않으면 아버지에게 이야기하겠다"고 협박했고, 협박에 돌아온 D양을 포항시 북구 한 모텔에 한 달 동안 감금한 혐의도 받고 있다. 이들은 당시 D양이 도망치지 못하도록 휴대폰에 위치추적 앱까지 설치해 관리했고, A씨는 연인인 C씨 몰래 D양을 4회에 걸쳐 성폭행한 혐의도 받는다.  C씨의 친오빠인 B씨는 2020년 3월 범행에 가담한 뒤 성매매를 거부하는 D양을 주먹과 발로 폭행하고, D양 핸드폰을 파손했다.        재판부는 "피해 청소년으로 하여금 성매매를 하게 하고, 그 수입을 지급받는 등 피해자를 경제적 이익 추구의 수단으로 삼았다는 점에서 그 죄책이 매우 무겁다"면서 "A씨는 위력으로 D양을 간음하는 범행까지 저질렀다"고 지적했다. 다만, "피고인들이 범행을 모두 인정

In [111]:
df_res[df_res.index == 4]['content']

4    \n\n\n\n\n가출 여고생에게 숙식을 제공하며 성매매를 강요한 일당이 실형을 선...
Name: content, dtype: object

이거 신기하네...  

In [112]:
df_res[df_res.index == 4].iloc[0]['content']

'\n\n\n\n\n가출 여고생에게 숙식을 제공하며 성매매를 강요한 일당이 실형을 선고받았다. 이들 중 한 명은 공범 몰래 여고생을 성폭행하기도 했다. 사진은 기사와 무관함. /사진=게티이미지뱅크 \n가출 여고생에게 숙식을 제공하며 성매매를 강요한 일당이 실형을 선고받았다. 이들 중 한 명은 공범 몰래 여고생을 성폭행하기도 했다. 대구지방법원 포항지원 제1형사부(권순향 재판장)는 아동청소년의성보호에관한법률위반(강요행위) 등의 혐의로 기소된 A씨(22), B씨(25), C씨(22·여)에게 각각 징역 6년, 징역 5년, 징역 4년을 선고했다고 19일 밝혔다.         재판부는 이들에게 80시간의 성폭력 치료프로그램 이수와 7년간 아동·청소년·장애인 취업 제한도 함께 명령했다. A씨 등은 2019년 11월부터 2020년 9월까지 고향 후배인 D양(17)에게 숙식을 제공하면서 성매매를 강요한 혐의를 받고 있다. 이들은 월 평균 25회, 하루 3~4회에 걸쳐 휴대폰 애플리케이션 등으로 성매수남을 모집해 D양에게 성매매를 강제했다. 또 D양이 성매매 대가로 받아돈 금액 중 70%를 갈취했다.        연인 관계인 A씨와 C씨는 2020년 8월 D양이 경기 인천으로 도망치자 "전화를 받지 않으면 아버지에게 이야기하겠다"고 협박했고, 협박에 돌아온 D양을 포항시 북구 한 모텔에 한 달 동안 감금한 혐의도 받고 있다. 이들은 당시 D양이 도망치지 못하도록 휴대폰에 위치추적 앱까지 설치해 관리했고, A씨는 연인인 C씨 몰래 D양을 4회에 걸쳐 성폭행한 혐의도 받는다.  C씨의 친오빠인 B씨는 2020년 3월 범행에 가담한 뒤 성매매를 거부하는 D양을 주먹과 발로 폭행하고, D양 핸드폰을 파손했다.        재판부는 "피해 청소년으로 하여금 성매매를 하게 하고, 그 수입을 지급받는 등 피해자를 경제적 이익 추구의 수단으로 삼았다는 점에서 그 죄책이 매우 무겁다"면서 "A씨는 위력으로 D양을 간음하는 범행까지 저질렀다"고 지적했다. 다만, "피고인들이 범행을 모두 인정

이러니까 다 나오네...  

#### inspection: continue

In [138]:
import re

def mark_keyword(text, keyword):
    # Define the pattern to search for the keyword, with word boundaries to match whole words only
    pattern = r'\b{}'.format(re.escape(keyword))

    # Use re.sub to replace the keyword with the keyword surrounded by parentheses
    marked_text = re.sub(pattern, r'🔺\g<0>🔻', text, flags=re.IGNORECASE)

    return marked_text

In [116]:
search = re.search(
    r'\b{}'.format(keyword),
    text,
    re.IGNORECASE)
search

<re.Match object; span=(217, 220), match='옥련동'>

In [118]:
search.span()

(217, 220)

In [143]:
idx = 112
row = df_res[df_res.index == idx]

# text = df_res[df_res.index == idx]['content'].to_string(index=False)
text = row.iloc[0]['content']
# print(text)

keyword = row.iloc[0][columns[1]]
print(keyword)

복대동


In [144]:
result = mark_keyword(text, keyword)
print(result)






화성연쇄살인 용의자 이춘재/사진=연합뉴스
화성연쇄살인사건의 유력한 용의자 이춘재(56)가 청주시와 수원시에서도 살인사건을 저질렀다고 자백했다.6일 경찰 등에 따르면 이춘재는 14건의 살인사건에 대한 자백을 내놨다. 화성사건은 8차 사건을 포함하면 모두 10건이다. 나머지 4건의 살인사건은 충북 청주와 수원에서 벌인 것으로 추정된다. 다만 이춘재의 자백에 대한 경찰의 신빙성 검증작업 병행이 필요할 전망이다.       우선 이춘재가 자백한 청주 살인사건은 2건이다. 그는 1991년 1월 27일 청주시 가경동 택지조성공사 현장 콘크리트관 속에서 속옷으로 입이 틀어막히고 양손을 뒤로 묶인 상태로 숨진 채 발견된 박모(17) 양 사건을 자신이 범행했다고 시인했다.당시 경찰은 박 양이 괴한에게 성폭행·살해당한 것으로 보고 3개월의 수사 끝에 박모(19) 군을 유력 용의자로 체포했지만, 증거 부족 등을 이유로 법원에서 무죄판결을 받으면서 이 사건은 미제로 남았다.1992년 6월 24일 🔺복대동🔻에서 발생한 가정주부 이모(28) 씨 피살사건도 이춘재의 범행으로 추정된다. 경찰은 당시 20대 초반으로 추정되는 남성이 사건 현장에서 나갔다는 목격자의 진술을 확보해 피해자와 남편 등 주변인을 중심으로 수사를 폈지만 끝내 사건을 해결하지 못했다.       1988∼1989년 연이어 터진 수원 여고생 살인사건도 이춘재의 소행으로 여겨진다. 수원 여고생 살인사건은 1987년 12월 24일 여고생이 어머니와 다투고 외출한 뒤 실종됐다가 열흘가량 뒤인 1988년 1월 4일 수원에서 숨진 채 발견된 사건이다. 6차와 7차 화성사건 사이에 벌어졌고 속옷으로 재갈이 물린 채 손이 결박된 상태로 발견돼 화성 사건과 유사성이 높다.




화성연쇄살인 용의자 이춘재/사진=연합뉴스
이듬해인 1989년 7월 3일 또 다른 여고생이 수원시 권선구 오목천동 야산 밑 농수로에서 흉기에 찔려 숨진 채 발견된 사건도 이 씨가 자백한 범행 중 1건으로 꼽힌다. 발생지역이 화성이 아니고 피해자의 손발

### todo: extract sentences

In [ ]:
# import re

# def mark_keyword(text, keyword):
#     # Define the pattern to search for the keyword, with word boundaries to match whole words only
#     pattern = r'\b{}\b'.format(re.escape(keyword))

#     # Use re.sub to replace the keyword with the keyword surrounded by parentheses
#     marked_text = re.sub(pattern, r'(\g<0>)', text, flags=re.IGNORECASE)

#     return marked_text

In [147]:
import re

def mark_keyword(text, keyword):
    # Define the pattern to search for the keyword, with word boundaries to match whole words only
    pattern = r'\b{}'.format(re.escape(keyword))

    # Use re.sub to replace the keyword with the keyword surrounded by parentheses
    marked_text = re.sub(pattern, r'🔺\g<0>🔻', text, flags=re.IGNORECASE)

    return marked_text

In [183]:
def extract_sentences_with_keyword(text, keyword):
    # Mark the keyword in the text
    marked_text = mark_keyword(text, keyword)

    # Split the text into sentences
    sentences = re.split(r'(?<=[.!?])+', marked_text)
    # sentences = re.split(r'(?<=[.!?]) +', marked_text)
    # sentences = re.split(r'🔺?<=[.!?]🔻 +', marked_text)
    o = [remove_linefeed(sentence).strip() for sentence in sentences]
    ic(o)

    # Filter the sentences that contain the marked keyword
    keyword_pattern = re.escape(f'🔺{keyword}🔻')
    keyword_sentences = [sentence for sentence in sentences if re.search(keyword_pattern, sentence, re.IGNORECASE)]

    return keyword_sentences

In [146]:
# Example usage
text = 'Blueberries are here, and they are purple. I love blueberry. Bananas are here, and they are yellow. I love bananas. Apples are here, and they are red. I love apples.'
keyword = 'bananas'

# Extract sentences containing the keyword
sentences_with_keyword = extract_sentences_with_keyword(text, keyword)

# Print the result
for sentence in sentences_with_keyword:
    print(sentence)

(Bananas) are here, and they are yellow.
I love (bananas).


In [161]:
def isexists_space(text:str) -> bool:
    return True if '  ' in text else False

def isexists_linefeed(text:str) -> bool:
    return True if '\n\n' in text else False

In [172]:
def remove_linefeed(text:str) -> str:
    return text.replace('\n', '')

In [189]:
def clean_text(text:str) -> str:
    flag = True
    while flag:
        flag_space = True if '  ' in text else False
        flag_linefeed = True if '\n\n' in text else False
        if flag_space or flag_linefeed: 
            if flag_space:
                text = text.replace('  ', ' ')
            if flag_linefeed:
                text = text.replace('\n\n', '\n')
        else:
            flag = False
    return text

In [168]:
idx = 112
row = df_res[df_res.index == idx]

# text = df_res[df_res.index == idx]['content'].to_string(index=False)
text = row.iloc[0]['content']
# print(text)
text = clean_text(text)

keyword = row.iloc[0][columns[1]]
print(keyword)

복대동


In [185]:
text

'\n화성연쇄살인 용의자 이춘재/사진=연합뉴스\n화성연쇄살인사건의 유력한 용의자 이춘재(56)가 청주시와 수원시에서도 살인사건을 저질렀다고 자백했다.6일 경찰 등에 따르면 이춘재는 14건의 살인사건에 대한 자백을 내놨다. 화성사건은 8차 사건을 포함하면 모두 10건이다. 나머지 4건의 살인사건은 충북 청주와 수원에서 벌인 것으로 추정된다. 다만 이춘재의 자백에 대한 경찰의 신빙성 검증작업 병행이 필요할 전망이다. 우선 이춘재가 자백한 청주 살인사건은 2건이다. 그는 1991년 1월 27일 청주시 가경동 택지조성공사 현장 콘크리트관 속에서 속옷으로 입이 틀어막히고 양손을 뒤로 묶인 상태로 숨진 채 발견된 박모(17) 양 사건을 자신이 범행했다고 시인했다.당시 경찰은 박 양이 괴한에게 성폭행·살해당한 것으로 보고 3개월의 수사 끝에 박모(19) 군을 유력 용의자로 체포했지만, 증거 부족 등을 이유로 법원에서 무죄판결을 받으면서 이 사건은 미제로 남았다.1992년 6월 24일 복대동에서 발생한 가정주부 이모(28) 씨 피살사건도 이춘재의 범행으로 추정된다. 경찰은 당시 20대 초반으로 추정되는 남성이 사건 현장에서 나갔다는 목격자의 진술을 확보해 피해자와 남편 등 주변인을 중심으로 수사를 폈지만 끝내 사건을 해결하지 못했다. 1988∼1989년 연이어 터진 수원 여고생 살인사건도 이춘재의 소행으로 여겨진다. 수원 여고생 살인사건은 1987년 12월 24일 여고생이 어머니와 다투고 외출한 뒤 실종됐다가 열흘가량 뒤인 1988년 1월 4일 수원에서 숨진 채 발견된 사건이다. 6차와 7차 화성사건 사이에 벌어졌고 속옷으로 재갈이 물린 채 손이 결박된 상태로 발견돼 화성 사건과 유사성이 높다.\n화성연쇄살인 용의자 이춘재/사진=연합뉴스\n이듬해인 1989년 7월 3일 또 다른 여고생이 수원시 권선구 오목천동 야산 밑 농수로에서 흉기에 찔려 숨진 채 발견된 사건도 이 씨가 자백한 범행 중 1건으로 꼽힌다. 발생지역이 화성이 아니고 피해자의 손발이 묶이지 않은 점 때문에 화성

In [184]:
result = extract_sentences_with_keyword(text, keyword)
print(result)

ic| o: ['화성연쇄살인 용의자 이춘재/사진=연합뉴스화성연쇄살인사건의 유력한 용의자 이춘재(56)가 청주시와 수원시에서도 살인사건을 저질렀다고 '
        '자백했다.',
        '6일 경찰 등에 따르면 이춘재는 14건의 살인사건에 대한 자백을 내놨다.',
        '화성사건은 8차 사건을 포함하면 모두 10건이다.',
        '나머지 4건의 살인사건은 충북 청주와 수원에서 벌인 것으로 추정된다.',
        '다만 이춘재의 자백에 대한 경찰의 신빙성 검증작업 병행이 필요할 전망이다.',
        '우선 이춘재가 자백한 청주 살인사건은 2건이다.',
        '그는 1991년 1월 27일 청주시 가경동 택지조성공사 현장 콘크리트관 속에서 속옷으로 입이 틀어막히고 양손을 뒤로 묶인 상태로 숨진 채 '
        '발견된 박모(17) 양 사건을 자신이 범행했다고 시인했다.',
        '당시 경찰은 박 양이 괴한에게 성폭행·살해당한 것으로 보고 3개월의 수사 끝에 박모(19) 군을 유력 용의자로 체포했지만, 증거 부족 '
        '등을 이유로 법원에서 무죄판결을 받으면서 이 사건은 미제로 남았다.',
        '1992년 6월 24일 🔺복대동🔻에서 발생한 가정주부 이모(28) 씨 피살사건도 이춘재의 범행으로 추정된다.',
        '경찰은 당시 20대 초반으로 추정되는 남성이 사건 현장에서 나갔다는 목격자의 진술을 확보해 피해자와 남편 등 주변인을 중심으로 수사를 '
        '폈지만 끝내 사건을 해결하지 못했다.',
        '1988∼1989년 연이어 터진 수원 여고생 살인사건도 이춘재의 소행으로 여겨진다.',
        '수원 여고생 살인사건은 1987년 12월 24일 여고생이 어머니와 다투고 외출한 뒤 실종됐다가 열흘가량 뒤인 1988년 1월 4일 '
        '수원에서 숨진 채 발견된 사건이다.',
        '6차와 7차 화성사건 사이에 벌어졌고 속옷으로 재갈

['1992년 6월 24일 🔺복대동🔻에서 발생한 가정주부 이모(28) 씨 피살사건도 이춘재의 범행으로 추정된다.']


## Test - 3

In [1]:
%load_ext autoreload

In [46]:
%aimport utils.search_address

In [47]:
import pandas as pd 
import numpy as np
import math
import re

from icecream import ic
from tqdm.notebook import tqdm 

from utils.custom_utils import load_var, save_var
from utils.search_address import RetrieveAddress, get_lat_long, CheckAddressLevels
from utils.webscraping import ArticleScraper
from utils.search_address import clean_text, extract_sentences_with_keyword, mark_keyword

from importlib import reload

df_addr = pd.read_parquet('./address/df_addr.parquet.gzip')
df_text = pd.read_parquet('./dataframe/flasher_hk_20130101_20220307.gzip')


In [48]:
%autoreload 2

### warming-up

In [6]:
df_text.head(1)

,범죄 유형,지역,위도,경도,중복 여부,퀄리티,제외 여부,기사제목,사건 장소,수사 기관,본문,URL,일자,언론사,기고자,비고,selenium_html,content
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""같이 살자"" 2억 뜯어낸 채팅녀 알고 보니 20대 남자였다",2억여원,"이대,검찰,한경닷컴 객원,서울서부지법,재판부,법원,형사항",채팅앱에서 만난 남성들에게 자신을 여성이라고 속인 뒤 2억여원을 가로챈 20대 남성...,https://www.hankyung.com/society/article/20220...,20220207,한국경제,이보배,NaN,"<body>\n\t<div id=""wrap"" class=""view"">\n\n\t\t...",\n\n\n\n\nA씨는 2020년 초부터 수 개월간 채팅앱에서 만난 남성들에게 자...


In [7]:
df_addr.head(1)

,lv0,lv1,lv2,lv3,lv4,lv5
0,충남,충청남도,천안시 동남구,대흥동,None,NaN


In [ ]:
text = df_text.iloc[0]['content']
text = ut.clean_text(text)

In [54]:
print(text.replace(".", ".\n"))

A씨는 2020년 초부터 수 개월간 채팅앱에서 만난 남성들에게 자신을 23살 여성이라고 소개한 뒤 함께 살자고 접근해 생활비 등의 명목으로 돈을 뜯어냈다.
 사진은 기사와 무관함.
 /사진=게티이미지뱅크 
채팅앱에서 만난 남성들에게 자신을 여성이라고 속인 뒤 2억여원을 가로챈 20대 남성이 항소심에서도 실형을 선고받았다.
 7일 법조계에 따르면 서울서부지법 형사항소2부(부상준 부장판사)는 사기·공갈·절도·여신전문금융업법 위반·컴퓨터등사용사기 등 혐의로 재판에 넘겨진 A씨(24)에게 1심과 같은 징역 5년을 선고했다.
 A씨는 2020년 초부터 수 개월간 채팅앱에서 만난 남성들에게 자신을 23살 여성이라고 소개했다.
 그는 남성들에게 교제를 하자거나 함께 살자고 접근해 생활비 등의 명목으로 돈을 뜯어낸 혐의로 재판에 넘겨졌다.
 A씨는 피해자 B씨에게 "같이 살 집을 구하자.
 보증금, 살림살이를 구입하는데 필요한 돈은 내가 관리하겠다"고 속여 2주 만에 3260만원을 편취했다.
 또 다른 피해자 C씨에게는 "사귀자"고 접근해 돈을 빌리고, 계좌번호와 비밀번호 등 개인정보를 건네 받아 은행에서 대신 대출받는 방식으로 총 1730만원을 빼앗았다.
 피해자 D씨에게는 음란행위 영상을 받은 뒤 "네가 일하는 곳에 영상을 뿌리겠다"고 협박해 410만원을 갈취하고, 수백만원을 대출받아 빼돌리기도 했다.
 1심 재판부는 "범행 수법과 방법 등이 상당히 불량하고, 사회적 비난의 여지가 큰 점 등을 불리한 정상으로 참작한다"면서 징역 5년을 선고했다.
 이후 A씨는 형이 너무 무겁다며, 검창른 너무 가볍다며 각각 항소했지만 항소심 재판부의 판단도 다르지 않았다.
 항소심 재판부는 "피해자가 수십 명이고, 피해액 합계가 2억4000만원에 이르는데도 피해 회복을 위해 별다른 노력을 하지 않았다"고 지적했다.
 이어 "절도 등으로 여러 차례 소년보호처분을 받은 전력이 있고, 재판 중에도 다른 미결수용자를 폭행하는 등 규율위반 행위로 금치 30일 처분을 받기도 했다"면서 "다만 잘못

In [78]:
cal = CheckAddressLevels(df_address=df_addr, text=text)
result = cal.process()
result

ic| key: '서울', v: '서울'
ic| key: '관리', v: '관리'
ic| key: '서울', v: '서울'


'관리'

In [63]:
extract_sentences_with_keyword(text=text, keyword=result.split(" ")[-1])

['보증금, 살림살이를 구입하는데 필요한 돈은 내가 🔺관리🔻하겠다"고 속여 2주 만에 3260만원을 편취했다.']

### step by step

In [67]:
cal.df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 693 entries, 2047 to 28198
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   lv0     693 non-null    object 
 1   lv1     693 non-null    object 
 2   lv2     693 non-null    object 
 3   lv3     693 non-null    object 
 4   lv4     10 non-null     object 
 5   lv5     0 non-null      float64
dtypes: float64(1), object(5)
memory usage: 37.9+ KB


In [88]:
cal = CheckAddressLevels(df_address=df_addr, text=text)
result = cal.process()
result

'관리'

In [94]:
cal.get_existance(verbose=True)

ic| '>>> get_existance: ', key: '관리', value: 1


ic| key: '관리'
ic| k: 'lv0', v: '서울', item_lv: 'lv4'
ic| k: 'lv1', v: '서울특별시', item_lv: 'lv4'
ic| k: 'lv2', v: '강동구', item_lv: 'lv4'
ic| k: 'lv3', v: '강일동', item_lv: 'lv4'
ic| k: 'lv4', v: '관리', item_lv: 'lv4'


................................................................................ 



ic| key: '서울'
ic| k: 'lv0', v: '서울', item_lv: 'lv4'
ic| 'key == v:', key: '서울', v: '서울'
ic| k: 'lv1', v: '서울특별시', item_lv: 'lv4'
ic| k: 'lv2', v: '강동구', item_lv: 'lv4'
ic| k: 'lv3', v: '강일동', item_lv: 'lv4'
ic| k: 'lv4', v: '관리', item_lv: 'lv4'


................................................................................ 

-------------------------------------------------------------------------------- 



...

ic| key: '관리'
ic| k: 'lv0', v: '서울', item_lv: 'lv0'
ic| k: 'lv1', v: '서울특별시', item_lv: 'lv0'
ic| k: 'lv2', v: '강동구', item_lv: 'lv0'
ic| k: 'lv3', v: '강일동', item_lv: 'lv0'
ic| k: 'lv4', v: '관리', item_lv: 'lv0'
ic| 'key == v:', key: '관리', v: '관리'


................................................................................ 



ic| key: '서울'
ic| k: 'lv0', v: '서울', item_lv: 'lv0'
ic| 'key == v:', key: '서울', v: '서울'
ic| k: 'lv1', v: '서울특별시', item_lv: 'lv0'
ic| k: 'lv2', v: '강동구', item_lv: 'lv0'
ic| k: 'lv3', v: '강일동', item_lv: 'lv0'
ic| k: 'lv4', v: '관리', item_lv: 'lv0'


................................................................................ 

-------------------------------------------------------------------------------- 



['관리', '서울']

In [72]:
len(cal.address_set)

717

In [74]:
cal.filtered_dict

{'관리': 1, '서울': 1}

In [75]:
cal.levels

{'관리': {'count': 1, 'level': ['lv4']}, '서울': {'count': 1, 'level': ['lv0']}}

In [76]:
cal.existances

['관리', '서울']

In [77]:
cal.existed_levels

{'관리': 4, '서울': 0}

### todo: modify counting and leveling

In [100]:
worktable = {
    "city": [],
    "count": [],
}

for key, value in cal.filtered_dict.items():
    worktable['city'].append(key)
    worktable['count'].append(value)

df_worktable = pd.DataFrame(worktable)
df_worktable

,city,count
0,관리,1
1,서울,1


In [127]:
for index, row in df_worktable.iterrows():
    ic(index, row)
    for column, value in row.items():
        ic(column, value)

ic| index: 0
    row: city     관리
         count     1
         Name: 0, dtype: object
ic| column: 'city', value: '관리'
ic| column: 'count', value: 1
ic| index: 1
    row: city     서울
         count     1
         Name: 1, dtype: object
ic| column: 'city', value: '서울'
ic| column: 'count', value: 1


#### count

In [104]:
import utils.search_address as sa

In [105]:
def make_address_from_columns(df:pd.DataFrame):
    address_set = set([])
    address_dict = {}
    for index, row in df.iterrows():
        for col, value in row.items():
            if isinstance(value, str):
                if value not in address_set:
                    address_set.add(value)
                    address_dict[value] = col
    return (address_set, address_dict)

In [106]:
addr_set, addr_dict = make_address_from_columns(cal.df_filtered)

In [107]:
# Initialize the dictionary with keywords and their counts set to 0
keyword_counts = {keyword: 0 for keyword in addr_set}

# Iterate through each keyword in the set
for keyword in addr_set:
    # Count the number of matches
    count = len(sa.re_findall(keyword, text))
    # Update the count for the keyword in the dictionary
    keyword_counts[keyword] += count

In [108]:
len(keyword_counts)

717

In [109]:
filtered_dict = {key: value for key, value in keyword_counts.items() if value > 0}
filtered_dict

{'관리': 1, '서울': 1}

In [110]:
df = cal.df_filtered.copy()

#### count - table

In [146]:
df.loc[:, 'count'] = 0
df.loc[:, 'found'] = ''
df.loc[:, 'found_lv'] = ''
# for index, row in df.iloc[:3].iterrows():
for index, row in df.iterrows():
    for column, value in row.items():
        if isinstance(value, str) and len(value) > 0:
            found = sa.re_findall(value, text)
            count = len(found)
            # ic(found, count)
            if count > 0:
                df.loc[index, 'count'] += count
                existed_value = df.loc[index, 'found']
                # df.loc[index, 'found'] = ','.join(found) if len(existed_value) == 0 else f"{existed_value},{','.join(found)}"
                df.loc[index, 'found'] = ','.join(found)
                df.loc[index, 'found_lv'] = cal.level_int[column]

In [138]:
found = sa.re_findall("a", "bababa ba tototo to")
result = ",".join(found)
result

''

In [147]:
df[df['count'] > 0]

,lv0,lv1,lv2,lv3,lv4,lv5,count,found,found_lv
2047,충남,충청남도,서천군,비인면,관리,NaN,1,관리,4
2838,충남,충청남도,태안군,이원면,관리,NaN,1,관리,4
4404,충북,충청북도,옥천군,이원면,관리,NaN,1,관리,4
4543,충북,충청북도,영동군,추풍령면,관리,NaN,1,관리,4
7732,경남,경상남도,하동군,적량면,관리,NaN,1,관리,4
...,...,...,...,...,...,...,...,...,...
28194,서울,서울특별시,강동구,둔촌동,None,NaN,1,서울,0
28195,서울,서울특별시,강동구,암사동,None,NaN,1,서울,0
28196,서울,서울특별시,강동구,성내동,None,NaN,1,서울,0
28197,서울,서울특별시,강동구,천호동,None,NaN,1,서울,0


In [115]:
df_addr.iloc[[2047, 2838, 4404, 4543]]

,lv0,lv1,lv2,lv3,lv4,lv5
2047,충남,충청남도,서천군,비인면,관리,NaN
2838,충남,충청남도,태안군,이원면,관리,NaN
4404,충북,충청북도,옥천군,이원면,관리,NaN
4543,충북,충청북도,영동군,추풍령면,관리,NaN


같은 동네 이름이 왜 이렇게 많아?  
'관리'는 도대체 뭐야?  

In [150]:
cal.make_count_table()

In [153]:
df = cal.df_filtered.copy()

In [155]:
df[df['found_lv'] > 2]

,lv0,lv1,lv2,lv3,lv4,lv5,count,found,found_lv
2047,충남,충청남도,서천군,비인면,관리,NaN,1,관리,4
2838,충남,충청남도,태안군,이원면,관리,NaN,1,관리,4
4404,충북,충청북도,옥천군,이원면,관리,NaN,1,관리,4
4543,충북,충청북도,영동군,추풍령면,관리,NaN,1,관리,4
7732,경남,경상남도,하동군,적량면,관리,NaN,1,관리,4
12963,전남,전라남도,고흥군,도양읍,관리,NaN,1,관리,4
17778,경북,경상북도,영천시,북안면,관리,NaN,1,관리,4
18743,경북,경상북도,청송군,파천면,관리,NaN,1,관리,4
25954,경기,경기도,이천시,마장면,관리,NaN,1,관리,4
26366,경기,경기도,화성시,향남읍,관리,NaN,1,관리,4


### todo: checking upper levels

In [1]:
%load_ext autoreload

In [19]:
%aimport utils.search_address

In [20]:
import pandas as pd 
import numpy as np
import math
import re

from icecream import ic
from tqdm.notebook import tqdm 

from utils.custom_utils import load_var, save_var
from utils.webscraping import ArticleScraper
from utils.search_address import CheckAddressLevels
# from utils.search_address import clean_text, extract_sentences_with_keyword, mark_keyword
import utils.search_address as sa

from importlib import reload

df_addr = pd.read_parquet('./address/df_addr.parquet.gzip')
df_text = pd.read_parquet('./dataframe/flasher_hk_20130101_20220307.gzip')


In [21]:
%autoreload 2

In [5]:
text = df_text.iloc[2]['content']
text = sa.clean_text(text)

#### warming-up

In [158]:
df_test = pd.DataFrame({'name':[], 'level':[], 'count':[]})
df_test

,name,level,count


In [164]:
df_test.loc[:, list(df_test.columns)] = ''
df_test

,name,level,count


In [167]:
df_test.loc[len(df_test.index)] = ['test', 0, 1]
df_test

,name,level,count
0,test,0,1
1,test,0,1


In [29]:
text = df_text.iloc[0]['content']
text = sa.clean_text(text)

In [11]:
cal = CheckAddressLevels(df_address=df_addr, text=text)
cal.process()

In [12]:
cal.df_brief

,name,level,count
0,관리,4,1
1,서울,0,1


#### ask: count table

In [13]:
import pandas as pd
import numpy as np
import re

class DataProcessor:
    def __init__(self, df_filtered, text, level_int):
        self.df_filtered = df_filtered
        self.text = text
        self.level_int = level_int
        self.address_set = set()
        self.df_brief = pd.DataFrame(columns=['name', 'level', 'count'])
    
    def make_count_table(self):
        df = self.df_filtered.copy()
        df['count'] = 0
        df[['found', 'found_lv']] = ''

        new_rows = []

        for index, row in df.iterrows():
            for column, value in row.items():
                if isinstance(value, str) and value:
                    found = re.findall(value, self.text)
                    count = len(found)
                    if count > 0:
                        df.at[index, 'count'] += count
                        df.at[index, 'found'] = ','.join(found)
                        df.at[index, 'found_lv'] = self.level_int[column]
                        
                        for item in found:
                            if item not in self.address_set:
                                self.address_set.add(item)
                                new_rows.append([item, self.level_int[column], count])
        
        self.df_brief = pd.concat([self.df_brief, pd.DataFrame(new_rows, columns=['name', 'level', 'count'])], ignore_index=True)
        self.df_filtered = df

In [15]:
# Example usage
df_filtered = pd.DataFrame({
    'A': ['pattern1', 'pattern2', '', 'pattern4'],
    'B': ['', 'pattern2', 'pattern3', '']
})
text = 'pattern1 pattern2 pattern2 pattern3 pattern4'
level_int = {'A': 1, 'B': 2}

processor = DataProcessor(df_filtered, text, level_int)
processor.make_count_table()

print(processor.df_filtered)
print("." * 80)
print(processor.df_brief)

          A         B  count              found found_lv
0  pattern1                1           pattern1        1
1  pattern2  pattern2      4  pattern2,pattern2        2
2            pattern3      1           pattern3        2
3  pattern4                1           pattern4        1
................................................................................
       name level count
0  pattern1     1     1
1  pattern2     1     2
2  pattern3     2     1
3  pattern4     1     1


#### apply

In [28]:
text

'pattern1 pattern2 pattern2 pattern3 pattern4'

In [30]:
cal = CheckAddressLevels(df_address=df_addr, text=text)
cal.process()

ic| item: '관리'
ic| item: '서울'


In [31]:
cal.df_brief

,name,level,count
0,관리,4,1
1,서울,0,1


#### get upper levels

In [34]:
','.join(cal.df_brief.columns)

'name,level,count'

In [38]:
df_upper = cal.df_brief[cal.df_brief['name'] == '관리']
df_upper

,name,level,count
0,관리,4,1


In [43]:
df_upper.loc[:, 'count'] = 2
df_upper

,name,level,count
0,관리,4,2


In [44]:
cal.df_brief = df_upper
cal.df_brief

,name,level,count
0,관리,4,2


In [45]:
cal = CheckAddressLevels(df_address=df_addr, text=text)
cal.process()

ic| item: '관리'


AttributeError: 'Series' object has no attribute 'columns'

Series 에서 컬럼 이름을 어떻게 알 수 있지?

In [48]:
df_upper.iloc[0].index

Index(['name', 'level', 'count'], dtype='object')

In [54]:
cal = CheckAddressLevels(df_address=df_addr, text=text)
cal.process()

ic| item: '관리'
ic| item: '서울'


In [56]:
cal.df_brief

,name,level,count,upper
0,관리,4,1,"lv0,lv1,lv2,lv3,lv4,lv5,count,found,found_lv"
1,서울,0,1,"lv0,lv1,lv2,lv3,lv4,lv5,count,found,found_lv"


아, 컬럼 이름이 아니지.  
컬럼 값이구나.  

In [57]:
df_upper.iloc[0].values

array(['관리', 4, 2], dtype=object)

In [58]:
values_set = set(df_upper.iloc[0].values)
values_set

{2, 4, '관리'}

In [62]:
values_set.update([2, 4, 2, 4])
values_set

{2, 4, '관리'}

In [65]:
cal.df_brief[cal.df_brief['name'] == '관리'].values

array([['관리', 4, 1, 'lv0,lv1,lv2,lv3,lv4,lv5,count,found,found_lv']],
      dtype=object)

In [68]:
[item if isinstance(item, str) else str(item) for item in values_set]

['관리', '2', '4']

#### dataframe: df_brief

In [6]:
text = df_text.iloc[0]['content']
text = sa.clean_text(text)

In [7]:
self = CheckAddressLevels(df_address=df_addr, text=text)
self.process()

In [8]:
self.df_brief

,name,level,count,upper
0,관리,4,1,"북안면,적량면,영천시,청송군,충북,충청남도,경기도,경상남도,이원면,태안군,전라남도,..."
1,서울,0,1,"현석동,삼선동2가,당산동3가,구완동,의주로2가,천호동,시흥동,북창동,신흥동2가,동숭..."


In [11]:
self.df_brief.loc[0, 'upper']

'북안면,적량면,영천시,청송군,충북,충청남도,경기도,경상남도,이원면,태안군,전라남도,옥천군,충청북도,전남,향남읍,비인면,추풍령면,경남,도양읍,영동군,하동군,충남,이천시,마장면,파천면,경상북도,경북,화성시,서천군,고흥군,관리,경기'

In [12]:
self.df_brief.loc[1, 'upper']

'현석동,삼선동2가,당산동3가,구완동,의주로2가,천호동,시흥동,북창동,신흥동2가,동숭동,을지로7가,남포동1가,상월곡동,정릉동,해안동3가,눌차동,양평동3가,구수동,홍파동,명륜1가,누하동,도봉동,주교동,남대문로2가,해안동1가,양화동,율목동,부평동4가,개봉동,갈월동,금호동3가,녹번동,서울특별시,신공덕동,천왕동,대림동,문래동5가,동산동,서대문구,일원동,필운동,교북동,동선동1가,상계동,수동,동광동1가,계산동2가,충정로2가,중동,원효로1가,부평동1가,북가좌동,창선동2가,대조동,동작구,역삼동,미음동,영등포동2가,우이동,명동2가,동소문동1가,가리봉동,저동1가,구의동,효자동,용동,명륜4가,양평동2가,운북동,통의동,제기동,회현동3가,미근동,마천동,명동1가,갈현동,목달동,항동,남포동4가,석관동,잠원동,이화동,생곡동,상암동,동자동,당산동6가,을지로5가,오쇠동,다운동,문래동2가,하월곡동,궁동,통인동,부사동,혜화동,오장동,충무로2가,대신동,호동,송월동3가,영등포동3가,안영동,소격동,송학동3가,이태원동,행당동,도원동,중구,시장북로,충무로1가,신대방동,면목동,문배동,광진구,성안동,성수동2가,항동1가,미아동,암사동,신계동,서계동,도렴동,구산동,무학동,문래동6가,송파동,순화동,길동,경운동,중림동,회현동2가,종로구,동소문동5가,도화동,어남동,수송동,사당동,종로2가,도선동,동소문동2가,양평동6가,수유동,서야동,삼성동,익선동,송월동,답동,영등포동8가,용산동3가,중앙동3가,을지로1가,신생동,장사동,신수동,용산동4가,양평동1가,적선동,서울,동문동,인사동,보문동2가,염리동,등촌동,석촌동,동소문동7가,보수동3가,원효로4가,송월동1가,구랑동,을지로4가,신길동,태화동,원효로2가,청암동,삼덕동1가,종로6가,성남동,대창동2가,북성동1가,은평구,연건동,삼선동1가,문래동3가,홍익동,대창동1가,번동,신창동,안암동2가,태평동,장충동1가,남창동,유천동,신창동1가,옥교동,옥천동,공항동,삼선동4가,관수동,서초동,항동2가,오곡동,누상동,필동3가,청파동1가,용덕동,북정동,방산동,남대문로3가,장교동,한남동,소공동,광장동,안암동5

#### only upper levels

In [13]:
df_brief = self.df_brief.copy()
df_brief

,name,level,count,upper
0,관리,4,1,"북안면,적량면,영천시,청송군,충북,충청남도,경기도,경상남도,이원면,태안군,전라남도,..."
1,서울,0,1,"현석동,삼선동2가,당산동3가,구완동,의주로2가,천호동,시흥동,북창동,신흥동2가,동숭..."


In [21]:
for index, row in df_brief[:1].iterrows():
    ic(list(row.values))
    # for column, value in row.items():

ic| list(row.values): ['관리',
                       4,
                       1,
                       '북안면,적량면,영천시,청송군,충북,충청남도,경기도,경상남도,이원면,태안군,전라남도,옥천군,충청북도,전남,향남읍,비인면,추풍령면,경남,도양읍,영동군,하동군,충남,이천시,마장면,파천면,경상북도,경북,화성시,서천군,고흥군,관리,경기']


In [28]:
self = CheckAddressLevels(df_address=df_addr, text=text)
self.process()

ic| self.level_int[column]: 4
ic| row.values[:self.level_int[column]]: array(['충남', '충청남도', '서천군', '비인면'], dtype=object)
ic| self.level_int[column]: 0
ic| row.values[:self.level_int[column]]: array([], dtype=object)


In [29]:
df_brief = self.df_brief.copy()
df_brief

,name,level,count,upper
0,관리,4,1,"북안면,적량면,영천시,청송군,충북,충청남도,경기도,경상남도,이원면,태안군,전라남도,..."
1,서울,0,1,


In [30]:
df_brief.loc[0, 'upper']

'북안면,적량면,영천시,청송군,충북,충청남도,경기도,경상남도,이원면,태안군,전라남도,옥천군,충청북도,전남,향남읍,비인면,추풍령면,경남,도양읍,영동군,하동군,충남,이천시,마장면,파천면,경상북도,경북,화성시,서천군,고흥군,경기'

#### ask: separating functions or simplifying expressions

In [31]:
import pandas as pd
import re

class DataProcessor:
    def __init__(self, df_filtered, text, level_int):
        self.df_filtered = df_filtered
        self.text = text
        self.level_int = level_int
        self.address_set = set()
        self.df_brief = pd.DataFrame(columns=['name', 'level', 'count', 'upper_levels'])
    
    def set_to_list_string_only(self, data: set) -> list:
        return [item for item in data if isinstance(item, str)]
    
    def find_and_count(self, value, text):
        found = re.findall(value, text)
        return found, len(found)
    
    def update_brief(self, found_items, column_level, row_values):
        new_rows = []
        for item in found_items:
            if item not in self.address_set:
                self.address_set.add(item)
                upper_levels = ','.join(self.set_to_list_string_only(row_values[:column_level]))
                new_rows.append([item, column_level, 1, upper_levels])
            else:
                for brief_row in new_rows:
                    if brief_row[0] == item:
                        values_set = set(brief_row[-1].split(','))
                        values_set.update(row_values[:column_level])
                        brief_row[-1] = ','.join(self.set_to_list_string_only(values_set))
        return new_rows
    
    def make_count_table(self):
        df = self.df_filtered.copy()
        df['count'] = 0
        df[['found', 'found_lv']] = ''

        all_new_rows = []

        for index, row in df.iterrows():
            for column, value in row.items():
                if isinstance(value, str) and value:
                    found_items, count = self.find_and_count(value, self.text)
                    if count > 0:
                        df.at[index, 'count'] += count
                        df.at[index, 'found'] = ','.join(found_items)
                        df.at[index, 'found_lv'] = self.level_int[column]
                        new_rows = self.update_brief(found_items, self.level_int[column], row.values)
                        all_new_rows.extend(new_rows)
        
        self.df_brief = pd.concat([
            self.df_brief,
            pd.DataFrame(
                all_new_rows,
                columns=['name', 'level', 'count', 'upper_levels']
            )], ignore_index=True)
        self.df_filtered = df

In [32]:
# Example usage
df_filtered = pd.DataFrame({
    'A': ['pattern1', 'pattern2', '', 'pattern4'],
    'B': ['', 'pattern2', 'pattern3', '']
})
text = 'pattern1 pattern2 pattern2 pattern3 pattern4'
level_int = {'A': 1, 'B': 2}

processor = DataProcessor(df_filtered, text, level_int)
processor.make_count_table()

print(processor.df_filtered)
print(processor.df_brief)

          A         B  count              found found_lv
0  pattern1                1           pattern1        1
1  pattern2  pattern2      4  pattern2,pattern2        2
2            pattern3      1           pattern3        2
3  pattern4                1           pattern4        1
       name level count upper_levels
0  pattern1     1     1     pattern1
1  pattern2     1     1     pattern2
2  pattern3     2     1    ,pattern3
3  pattern4     1     1     pattern4


#### apply: 

In [5]:
text = df_text.iloc[16]['content']
text = sa.clean_text(text)

In [114]:
text_iter = iter(zip(df_text.index[10:], df_text.loc[10:, 'content']))

In [ ]:
index, text = next(text_iter)
text = sa.clean_text(text)

In [11]:
self = CheckAddressLevels(df_address=df_addr, text=text)
self.process()

df_brief = self.df_brief.copy()
print(df_brief)

for item in df_brief['name'].values:
    text = sa.re_sub(item, text)

ic| row_values: array(['인천', '인천광역시', '미추홀구', '숭의동', None, nan, 0, '', ''], dtype=object)
ic| o: ['인천', '인천광역시']
ic| found_items: ['미추홀구']
ic| upper_levels: []


   name level count upper
0    인천     0     5      
1  미추홀구     2     1      
2   부평구     2     1      


In [7]:
# print(f'\nindex: {index}\n')
print(text)

지적장애가 있는 여고생을 모텔에서 집단 폭행한 혐의를 받는 10대 A양과 B양이 지난 6월 28일 구속 전 피의자 심문(영장실질심사)을 받기 위해 🔺인천🔻시 🔺미추홀구🔻 🔺인천🔻지방법원에 들어서고 있다. /사진=연합뉴스
지적장애 3급 여고생을 모텔에서 집단 폭행하고 가학적인 행위를 일삼은 10대 청소년 등 5명에게 검찰이 징역형을 구형했다.🔺인천🔻지검은 26일 🔺인천🔻지법 형사9단독 결심 공판에서 폭력행위 등 처벌에 관한 법률 위반(공동상해) 등 혐의로 구속기소 된 A(17)양에게 징역 장기 5년, 단기 3년을 구형했다. 함께 피해자를 폭행하거나 방조한 혐의를 받는 다른 10대 남녀 2명에게는 단기 1년·장기 2년을, 함께 기소된 20대 남성에게는 징역 2년 형을 선고해달라고 재판부에 요청했다.이날 검찰은 "이들은 피해자를 옷을 벗겨 오물을 묻히는 등 가학적인 행위를 했으며, 같은 범죄 전력이 있는 가해자도 있다"며 "큰 충격을 받은 피해자와 그 가족이 엄벌을 원하고 있다"고 밝혔다.A 양 등은 지난 6월 16일 🔺인천🔻시 🔺부평구🔻 모텔에서 지적장애 3급인 16살 피해자를 폭행해 얼굴 등을 크게 다치게 한 혐의로 기소됐다. 당시 딸과 연락이 닿지 않자 휴대전화 애플리케이션으로 위치를 확인하고 모텔로 찾아간 피해자 어머니는 알몸 상태로 오물을 뒤집어쓴 딸을 발견해 경찰에 신고했다.A 양 등은 경찰 조사에서 "피해자가 자신들의 험담을 하고 다닌다고 생각해 범행했다"고 진술한 것으로 알려졌다.이날 A양 등은 최후 진술을 통해 "피해자와 그의 가족에게 상처를 줘 죄송하다"며 말했다. 한편, 이들의 선고 공판은 9월 중 열릴 예정이다.김정호 한경닷컴 객원기자 newsinfo@hankyung.com


In [16]:
text_iter = iter(zip(df_text.index[17:], df_text.loc[17:, 'content']))

In [22]:
index, text = next(text_iter)
text = sa.clean_text(text)

self = CheckAddressLevels(df_address=df_addr, text=text)
self.process()

df_brief = self.df_brief.copy()
print(df_brief)

text_mark = text
for item in df_brief['name'].values:
    text_mark = sa.re_sub(item, text_mark)

print(f'\nindex: {index}\n')
print(text_mark)

  name level count upper
0   충남     0     1      
1   부산     0     2      

index: 22

게티이미지뱅크
🔺부산🔻 한 남자고등학교 명의의 온라인 커뮤니티에서 인근 몇몇 여자고등학교 학생들을 성희롱한 게시글과 댓글이 올라와 경찰이 수사 중이다.🔺부산🔻 사하경찰서는 인근 여자 고등학생을 대상으로 성희롱 글을 학교 인터넷 커뮤니티에 올린 게시자를 찾는 한편 이들에 대한 명예훼손 혐의 적용 여부 등을 검토 중이라고 26일 밝혔다. 문제의 게시판에는 인근 몇몇 여자 고등학교의 코로나19 백신 접종일을 공유한 뒤, “OO일에 보건소 앞에서 보자”는 협박성 게시글과 함께 “화이자 백신인 줄 알고 맞은 것이 정자였다”는 등의 표현을 포함해 입에 담기 힘든 수위의 성희롱 댓글이 달렸던 것으로 알려졌다.이에 인근 여고 피해 학생들이 불안을 호소하며 학교, 교육청 등에 민원을 제기한 것으로 알려졌다. 현재 해당 게시글은 삭제된 상태다.인근 여고 학생들에 따르면 이번과 같은 사건이 처음이 아닌 것으로 전해진다. 여성 청소년과 교사 등을 겨냥한 남성 청소년의 사이버 성폭력이 심각한 수준이라는 지적이다. 이 남고 갤러리에는 이번 일이 있기 전에도 여교사 외모 평가, “특정 여고에 원조교제하는 애들이 있다” 등 모욕성 발언이 다수 올라왔던 것이 확인됐다. 앞서 지난 7일 🔺충남🔻경찰청 사이버범죄수사대는 한 남자고등학교 이름으로 된 인터넷 커뮤니티에 특정 여자 고등학생들을 성희롱한 학생 2명을 입건한 바 있다.안혜원 한경닷컴 기자 anhw@hankyung.com


- 4
    - 관리, 북구, 인천, 포항시 북구, 대구, 경기
    - 포항시 북구...
- 8  
    - 서울 '구로'경찰서  
    - '시'와 '구'는 이름만으로도 찾을 수 있어야 하겠다.
- 10
    - 춘천경찰서
- 13
    - 강북경찰서
- 15
    - '영등포구' 상위레벨로 '서울'이 나오지 않음.
    - 아, 본문에 '서울'이 없구나.
- 16
    - 인천 미추홀구, 상위레벨이 본무에 있는데도 나오지 않는다.
    - check upper levels after making the count table
    - 카운트 테이블을 만들 때에는, 텍스트에서 찾은 주소들을 모두 알 수 없기 때문에, 상위 레벨을 확인 하는 작업은 나중에 해야한다.  
- 17
    - 경북 예천의 ...
    - '예천'시 를 잡아내지 못함.
- 18
    - '법리' 검토가 필요...
- 21
    - '관리'에 관한 법률...
    - '춘천'지법 읽어내지 못함.
- 22
    - '부산' 사하경찰서 - '사하'구를 잡지 못함

#### ask: modifying re expression

In [27]:
import re

def mark_cherry(text: str) -> str:
    """
    Finds all occurrences of the word 'cherry' in the text and marks them with '>cherry<',
    but skips those that are already marked.

    Parameters:
    text (str): The input text.

    Returns:
    str: The text with 'cherry' marked as '>cherry<'.
    """
    # Regular expression to find 'cherry' that is not already marked with '>'
    pattern = r'(?<!🔺)\bcherry\b(?!🔻)'
    
    # Replace all found occurrences with '>cherry<'
    marked_text = re.sub(pattern, r'🔺cherry🔻', text, flags=re.IGNORECASE)
    
    return marked_text

In [28]:
# Example usage
text = "I love cherry pie. Have you tried 🔺cherry🔻 jam? cherry is my favorite fruit."
result = mark_cherry(text)
print(result)

I love 🔺cherry🔻 pie. Have you tried 🔺cherry🔻 jam? 🔺cherry🔻 is my favorite fruit.


#### apply

In [ ]:
text = df_text.loc[22, 'content']
text = sa.clean_text(text)
text_mark = text

In [42]:
for item in df_brief['name'].values:
    text_mark = sa.re_sub(item, text_mark)

print(f'\nindex: {index}\n')
print(text_mark)


index: 22

게티이미지뱅크
🔺부산🔻 한 남자고등학교 명의의 온라인 커뮤니티에서 인근 몇몇 여자고등학교 학생들을 성희롱한 게시글과 댓글이 올라와 경찰이 수사 중이다.🔺부산🔻 사하경찰서는 인근 여자 고등학생을 대상으로 성희롱 글을 학교 인터넷 커뮤니티에 올린 게시자를 찾는 한편 이들에 대한 명예훼손 혐의 적용 여부 등을 검토 중이라고 26일 밝혔다. 문제의 게시판에는 인근 몇몇 여자 고등학교의 코로나19 백신 접종일을 공유한 뒤, “OO일에 보건소 앞에서 보자”는 협박성 게시글과 함께 “화이자 백신인 줄 알고 맞은 것이 정자였다”는 등의 표현을 포함해 입에 담기 힘든 수위의 성희롱 댓글이 달렸던 것으로 알려졌다.이에 인근 여고 피해 학생들이 불안을 호소하며 학교, 교육청 등에 민원을 제기한 것으로 알려졌다. 현재 해당 게시글은 삭제된 상태다.인근 여고 학생들에 따르면 이번과 같은 사건이 처음이 아닌 것으로 전해진다. 여성 청소년과 교사 등을 겨냥한 남성 청소년의 사이버 성폭력이 심각한 수준이라는 지적이다. 이 남고 갤러리에는 이번 일이 있기 전에도 여교사 외모 평가, “특정 여고에 원조교제하는 애들이 있다” 등 모욕성 발언이 다수 올라왔던 것이 확인됐다. 앞서 지난 7일 🔺충남🔻경찰청 사이버범죄수사대는 한 남자고등학교 이름으로 된 인터넷 커뮤니티에 특정 여자 고등학생들을 성희롱한 학생 2명을 입건한 바 있다.안혜원 한경닷컴 기자 anhw@hankyung.com


#### 시, 구 이름만으로도 찾기

어떻게 시작해야할까?  
주소에서 '시', '구'를 제외한 컬럼을 만들까?  
정규식 패턴으로 할까?  

#### ask:

```
def find_all_things(text: str, keyword: str) -> list:
    pattern = r'\b(\w+){keyword}\b'
    matches = re.findall(pattern, text, flags=re.IGNORECASE)
    return matches


if a keyword contains 'thing' at the end then ignore 'thing'. e.g 'onething' to 'one', 'something' to 'some' ...
```

In [43]:
import re

def find_all_things(text: str, keyword: str) -> list:
    """
    Finds all occurrences of words that match the keyword pattern in the text.
    If the keyword ends with 'thing', the 'thing' part is ignored in the result.

    Parameters:
    text (str): The input text.
    keyword (str): The keyword pattern to search for in the text.

    Returns:
    list: A list of all matches found in the text.
    """
    # Remove 'thing' from the end of the keyword if it exists
    if keyword.endswith('thing'):
        keyword = keyword[:-5]
    
    # Regular expression pattern to match any word followed by 'thing'
    pattern = rf'\b{keyword}(\w*)thing\b'
    
    # Find all matches in the text
    matches = re.findall(pattern, text, flags=re.IGNORECASE)
    
    return matches

In [44]:
# Example usage
text = "I saw onething, twothing, manything, and something at the market. But no nothing or everything."
keyword = 'thing'
result = find_all_things(text, keyword)
print(result)

['one', 'two', 'many', 'some', 'no', 'every']


In [57]:
def isendswith(text: str, postfix: list) -> str:
    for item in postfix:
        if text.endswith(item):
            return item
    return ''


def find_all_things(text: str, keyword: str) -> list:
    end_word = isendswith(keyword, ['시', '구'])
    ic(end_word)
    if end_word:
        keyword = keyword[:-1]
        ic(keyword)
    # pattern = rf'\b{keyword}(\w*){end_word}'
    pattern = rf'\b{keyword}'
    ic(pattern)
    # matches = re.findall(pattern, text, flags=re.IGNORECASE)
    matches = re.findall(pattern, text)
    return matches

In [58]:
text = "양천구에서 무지개가 피어났다."
keyword = '양천구'
result = find_all_things(text, keyword)
print(result)

ic| end_word: '구'
ic| keyword: '양천'
ic| pattern: '\\b양천'


['양천']


#### apply

In [5]:
text = df_text.iloc[16]['content']
text = sa.clean_text(text)

In [6]:
self = CheckAddressLevels(df_address=df_addr, text=text)
self.process()

df_brief = self.df_brief.copy()
print(df_brief)

for item in df_brief['name'].values:
    text = sa.re_sub(item, text)

  name level count     upper
0    남     2     2  광주,광주광역시
1   인천     0     5          
2    중     2     1  인천광역시,인천
3  미추홀     2     1  인천광역시,인천
4   부평     2     1  인천광역시,인천


'남구', '중구' 처럼 외자 이름들이 있구나.  
예외가 정말 많다.  

In [24]:
text = df_text.iloc[16]['content']
text = sa.clean_text(text)

In [25]:
self = CheckAddressLevels(df_address=df_addr, text=text)
self.process()

df_brief = self.df_brief.copy()
print(df_brief)

ic| found_items: ['미추홀']
ic| found_items: ['미추홀']
ic| found_items: ['미추홀']
ic| found_items: ['미추홀']
ic| found_items: ['미추홀']
ic| found_items: ['미추홀']
ic| found_items: ['미추홀']


  name level count     upper
0   인천     0     5          
1  미추홀     2     1  인천광역시,인천
2   부평     2     1  인천광역시,인천


뭐야, 왜 안나와? 왜 비었어?

In [23]:
self.df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   lv0       0 non-null      object 
 1   lv1       0 non-null      object 
 2   lv2       0 non-null      object 
 3   lv3       0 non-null      object 
 4   lv4       0 non-null      object 
 5   lv5       0 non-null      float64
 6   count     0 non-null      int64  
 7   found     0 non-null      object 
 8   found_lv  0 non-null      object 
dtypes: float64(1), int64(1), object(7)
memory usage: 0.0+ bytes


아, 텍스트 설정을 안 했구나.

In [ ]:
for item in df_brief['name'].values:
    text = sa.re_sub(item, text)

In [16]:
'basil leaves'.find('leaf')

-1

In [25]:
self = CheckAddressLevels(df_address=df_addr, text=text)
self.process()

df_brief = self.df_brief.copy()
print(df_brief)

ic| found_items: ['미추홀']
ic| found_items: ['미추홀']
ic| found_items: ['미추홀']
ic| found_items: ['미추홀']
ic| found_items: ['미추홀']
ic| found_items: ['미추홀']
ic| found_items: ['미추홀']


  name level count     upper
0   인천     0     5          
1  미추홀     2     1  인천광역시,인천
2   부평     2     1  인천광역시,인천


되었다. 이제 다시 결과를 살펴보자.  

#### inspection

이쯤에서 새 노트로 옮겨가야겠다.

## Get Coodinates

In [27]:
get_lat_long(addr)

(37.4782, 126.9518)

# end